# <h1 style="text-align:center;font-size:36px">***GRAPH DES DONNEES DE COMPTAGES***</h1>
> il y a un exemple de verification si doublons de point dans le 17, 47, 87<br>

|Gest|Gest|Gest|
|-|-|-|
|[CD23](#1)|[CD64](#5)|[DIRSO](#9)|
|[CD40](#2)|[ASF](#6)|[Anglet](#10)|
|[CD17](#3)|[DIRA](#7)||
|[Alienor / Atlandes](#4)|[DIRCO](#8)||

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
import altair as alt
import csv, re, os, statistics, filecmp, unidecode
import datetime as dt
from math import sqrt, pi, exp, log, log10
import Connexion_Transfert as ct
import Import_trafics as it
from Donnees_horaires import comparer2Sens,verifValiditeFichier,concatIndicateurFichierHoraire,SensAssymetriqueError,verifNbJoursValidDispo, tmjaDepuisHoraire, periodeDepuisHoraire, attributsHoraire
from Donnees_sources import MHCorbin, NettoyageTemps, GroupeCompletude, NombreDeJours
from Params.DonneesSourcesParams import MHcorbinMaxLength, MHcorbinMaxSpeed, MHCorbinValue0, MHCorbinFailAdviceCode
import Outils as O
from collections import Counter
from shapely.geometry import Point,LineString, MultiLineString
from shapely import speedups
speedups.disable()
from shapely.ops import transform
from geoalchemy2 import Geometry, WKTElement
#from Base_BdTopo import Import_outils as io
#from Base_BdTopo import Rond_points as rp#from Base_BdTopo import Regroupement_correspondance as rc
from sqlalchemy.schema import MetaData
from sqlalchemy import inspect
from statistics import mean
from itertools import combinations
from unidecode import unidecode

from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import pyproj
from shapely.ops import transform

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 30)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id="1" ></a>
# ***CD 23***
- Année 2019 : 
> ***attention : pour le point de comptage D941 6+152 à Aubusson, le pR est 32 et non 6. il faut donc corriger à la main le fihcier excel***
<br> Pour le moment tous les points sont déjà dans  la base, dc pas de traitement de type insert prévus.

In [ ]:
#initiliser les données
cd23=it.Comptage_cd23(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD23\2019-CD23_trafics.xls',2019)
df_propre=cd23.ouvrirMiseEnForme()
cd23.classer_comptage_update_insert('gti_otv_pg11', 'na_2010_2019_p')
cd23.update_bdd_23('gti_otv_pg11','comptage', 'na_2010_2019_p')
cd23.donneesMens()
cd23.insert_bdd('gti_otv_pg11','comptage', 'na_2010_2019_mensuel',cd23.df_attr_mensuel)

<a id="2" ></a>
# ***CD 40***
> 2019 : envoi des données B152 de route +   
2020 : envoi des données B153 de route +

In [ ]:
# 2019 : initialiser
cd40=it.Comptage_cd40(r'D:\temp\otv\2019\Donnees_source\CD40\Observatoire des trafics')
#mettre enf orme
cd40.comptage_forme()
cd40.classer_comptage_insert_update('local_otv_station_gti', 'na_2010_2019_p', 'comptage')
#transfert donnees_agregees
cd40.update_bdd_40('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
#donnees_mensuelles
#cd40.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',cd40.df_attr_mens)

In [ ]:
#2020 : preparer
cd40=it.Comptage_cd40(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\CD40\en_cours\donnees', donneesType='B153', annee='2020')
cd40.comptage_forme()
cd40.classer_comptage_insert_update('local_otv_boulot', 'compteur', 'comptage')
#cd40.df_attr_update,cd40.df_attr_insert, cd40.df_attr_mens

#mettre a jour 
#comptages
dfComptage=cd40.creer_comptage(cd40.df_attr_update.id_comptag.tolist(), cd40.annee, 'tableur CD40', 'tv/pl', obs='fichier B153 route +')
cd40.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dfComptage)
#indics_agreges
dfIndicAgrege=cd40.structureBddOld2NewForm(cd40.df_attr_update, cd40.annee,['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl', 'tmje', 'tmjhe'], 'agrege')
cd40.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege', dfIndicAgrege)
#trafic mensuel
dfIndicMensuel=cd40.structureBddOld2NewForm(cd40.df_attr_mens, cd40.annee,['id_comptag', 'annee', 'fichier', 'donnees_type'],['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept',
       'octo', 'nove', 'dece'], 'mensuel')
cd40.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel', dfIndicMensuel)

<a id="3" ></a>
# ***CD17***

> ### Année 2015 : traitée dans le fichier Import_trafics.py

> ### année 2016 données issue des borchures de comptage, uniquemnet pour ponctuels

In [ ]:
#ouverture du fichier
cpt17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_produites\donnnees_travail\Comptage\17\cpt_pctuel_but_2016.txt',
                       'brochure',2016)

#mettre à jour les id_comptage deja presents (attention, update Bdd à changé, il faut passer par creer_valeur_txt_update avant et certains paramètres ont changé)
cpt17.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2017_p', '17','ponctuel') #creer les attributs selon les donnees presentes dans la base
cpt17.update_bdd('gti_otv', 'comptage', 'na_2010_2017_p')#mise à jour

#creer referentiel si besoin
with ct.ConnexionBdd('gti_otv') as c:
    ct.ogr2ogr_shp2pg(c.connstringOgr, r'Y:\REF_GEO\BD_Topo\D17\ED16\SHP\1_DONNEES_LIVRAISON\N_TRONCON_ROUTE_BDT_017.SHP',
                     schema='referentiel',table='troncon_route_bdt17_ed16_l',geotype='MULTILINESTRING', dims=2, encodageClient='LATIN1' )
#creer graph
rqt="""
alter table referentiel.troncon_route_bdt17_ed16_l add column source integer , add column target integer ;
select pgr_createTopology ('referentiel.troncon_route_bdt17_ed16_l',1,'geom', 'ogc_fid') ;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN id TO id_ign;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN ogc_fid TO id;
alter table referentiel.troncon_route_bdt17_ed16_l add column long_km numeric ;
update referentiel.troncon_route_bdt17_ed16_l set long_km=(st_length(geom)/1000) ;
""" #attention, il manque la ligne au dessus pour créer l'analyseGraph qui va renvoyer le nb de count
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

#inserer les nouveaux comptages
with ct.ConnexionBdd('gti_otv') as c:
    cpt17.df_attr_insert.to_sql('na_2010_2017_p',c.sqlAlchemyConn,schema='comptage',if_exists='append', index=False )

#mettre à jour la geom 
rqt=""" update comptage.na_2010_2017_p
  set geom=(select geom_out  from comptage.geoloc_pt_comptag(id_comptag))
  where dep='17' and geom is null
"""
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

> ### fichier compteurs permanents format csv annee 2017, 2018, 2019

In [28]:
cpt_perm=it.Comptage_cd17(r'D:\Boulot\AffairesEnCours\OTV\17\csv perso pactol_2019_1.csv',
                         'permanent_csv',2019)
cpt_perm.mises_forme_bdd_brochure_pdf('local_otv_gti', 'comptage', 'na_2010_2019_p', '17','permanent', 'maison')
#miettre à jour les données deja existantes
cpt_perm.update_bdd_17('local_otv_gti', 'comptage', 'na_2010_2019_p')#mise à jour
#inseérer les données nouvelles
cpt_perm.insert_bdd('gti_otv', 'comptage', 'na_2010_2019_p')
#mettre à jour la geom 
cpt_perm.maj_geom('gti_otv', 'comptage', 'na_2010_2019_p', '17')
# pour les données mensuelles
cpt_perm.insert_bdd_mens('local_otv_gti', 'comptage','na_2010_2019_mensuel')

> ### annee 2018 ; fichier compteurs tournant format excel issu des donnees pour brochure

In [ ]:
#initiliser la classe avec le fichier
bdd='gti_otv_pg11'
cpt_cd17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\Brochure 2018 CD17 DREAL\10  5 1 B3 tournants recalculés.xls',
                      'tournant_xls_bochure',2018)

#mise en forme des données
cpt_cd17.comptag_existant_bdd(bdd, 'na_2010_2018_p', dep='17')
donnees=cpt_cd17.ouvrir_xls_tournant_brochure()
cpt_cd17.conversion_id_comptg_existant_xls_brochure(bdd)
cpt_cd17.carac_xls_brochure()

#mise à jour des données
val_txt=cpt_cd17.creer_valeur_txt_update(cpt_cd17.df_attr_update, ['id_comptag','tmja_2018','tmja_2017'])
cpt_cd17.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja_2018','tmja_2017':'tmja_2017'})

### annee 2017-2018 ; fichier ponctuel excel qui alimente des brochures

In [ ]:
#ouvrir le fichier et initialisation
cpt_pct2018_cd17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\Brochure 2018 CD17 DREAL\spécifique Brochure V85_2018 .xls',
                      'ponctuel_xls_bochure',2018)
bdd='gti_otv_pg11'
#mise en forme
cpt_pct2018_cd17.comptag_existant_bdd(bdd, 'na_2010_2018_p', dep='17')
cpt_pct2018_cd17.conversion_id_comptg_existant_xls_brochure(bdd)
cpt_pct2018_cd17.filtrer_periode_ponctuels_xls_brochure()
cpt_pct2018_cd17.carac_xls_brochure()

#mise à jour des données
val_txt=cpt_pct2018_cd17.creer_valeur_txt_update(cpt_pct2018_cd17.df_attr_update, ['id_comptag','tmja','pc_pl','obs'])
cpt_pct2018_cd17.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja','pc_pl_2018':'pc_pl', 'obs_2018':'obs'})

In [ ]:
#trouver les points de comptages a inserer situe sur le même troncon elementaires que d'autres points
table_corresp=cpt_pct2018_cd17.correspondance_ancien_nouveau_comptage(bdd,'public','cd17_tournant_insert','lineaire.traf2016_bdt17_ed16_l',
                                                    'public', 'traf2016_bdt17_ed16_l','traf2016_bdt17_ed16_l_vertices_pgr')

#si le point fait partie de la table_corresp, on insère pas, sinon on insère
pt_a_inserer=cpt_pct2018_cd17.df_attr_insert.loc[~cpt_pct2018_cd17.df_attr_insert.id_comptag.isin(table_corresp.id_comptag.tolist())].copy()
#mise en form avant insertion
pt_a_inserer['dep']='17'
pt_a_inserer['route']=pt_a_inserer.id_comptag.apply(lambda x : x.split('-')[1])
pt_a_inserer.rename(columns={'absc':'abs','tmja':'tmja_2018','pc_pl':'pc_pl_2018','obs':'obs_2018'},inplace=True)
pt_a_inserer['reseau']='RD'
pt_a_inserer['gestionnai']='CD17'
pt_a_inserer['concession']='N'
pt_a_inserer['type_poste']='ponctuel'
pt_a_inserer=pt_a_inserer[['id_comptag','dep','route','pr','abs','reseau','gestionnai','concession','type_poste','tmja_2018','pc_pl_2018','obs_2018']].copy()
#si plusieurs fois le mm point on garde la valeur max
pt_a_inserer=pt_a_inserer.loc[pt_a_inserer.tmja_2018==pt_a_inserer.groupby('id_comptag').tmja_2018.transform(max)].copy()

cpt_pct2018_cd17.insert_bdd(bdd, 'comptage', 'na_2010_2018_p', pt_a_inserer)
cpt_pct2018_cd17.maj_geom(bdd, 'comptage', 'na_2010_2018_p', dep='17')

# ***CD19***
> 2019 : creation de la classe Comptage_cd19

<a id="5" ></a>
# ***CD64***

In [80]:
#init
fichier=r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\CD64\en_cours\Copie de SIG TV PL %PL TOURNANTS + PERMANENTS 2020.xlsx'
cd64=it.Comptage_cd64(fichier, '2020')
cd64.miseEnForme()
cd64.classer_comptage_insert_update()

In [83]:
#verfier les correspondances
#verifieles geometries a insrere
#calcul auto des geometries et verif Qgis
gdfInsert=cd64.localiser_comptage_a_inserer(cd64.df_attr_insert, 'local_otv_boulot', 'public', 'localiser_pt_cd64', 
                                            'ref.troncon_route_bdt_na_ed20_l', 'ref.pr_ed18_p')
#gdfInsert.to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\verif_pt_cd64.shp')
#reprendre le fichier corrige via Qgis
gdfInsertCorrigee=gp.read_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\verif_pt_cd64.shp')
dicoCoresp={'64-D10-3+0':'64-D10-3+400','64-D4-25+0':'64-D4-25+500', '64-D+6-8+130':'64-D55-2+535', '64-D6-2+335':'64-D6-0+250'}
cd64.verifComptageInsert(dicoCoresp,gdfInsertCorrigee)

In [162]:
#mise à jour des données déjà présentes : 
#mise a jour de obs_supl
cd64.update_bdd('comptage', 'compteur', cd64.creer_valeur_txt_update(cd64.df_attr_update, ['id_comptag', 'obs_supl']), {'obs_supl':'obs_supl'})
#insere les nouveaux comptages, mettre à jour les anciens
#séparer les comptages deja existants et les nouveaux
bdd='local_otv_boulot'
with ct.ConnexionBdd(bdd) as c  :
    comptagesExistant=pd.read_sql('select ca.* from comptage.comptage ca join comptage.compteur ce on ca.id_comptag=ce.id_comptag where ce.dep=\'64\'', c.sqlAlchemyConn)
dfJointureBddCd64=cd64.df_attr_update.assign(annee=cd64.df_attr_update.annee.astype(str)).merge(comptagesExistant[['id','id_comptag', 'annee']], how='left', on=['id_comptag', 'annee'])
dfComptagesExistants=dfJointureBddCd64.loc[~dfJointureBddCd64.id.isna()]
dfComptagesNew=dfJointureBddCd64.loc[dfJointureBddCd64.id.isna()]
#mise à jour des existants
#comptage
cd64.update_bdd('comptage', 'comptage', cd64.creer_valeur_txt_update(dfComptagesExistants[['id']].assign(src='tableur CD64'), ['id', 'src']),
                {'src':'src'}, identifiant='id')
#tmja
cd64.update_bdd('comptage', 'indic_agrege', cd64.creer_valeur_txt_update(dfComptagesExistants.assign(id_comptag_uniq=dfComptagesExistants.id)
                                                                         , ['id_comptag_uniq', 'tmja']), {'valeur':'tmja'}, identifiant='id_comptag_uniq',
                filtre = "indicateur='tmja'")
#pc_pl
cd64.update_bdd('comptage', 'indic_agrege', cd64.creer_valeur_txt_update(dfComptagesExistants.assign(id_comptag_uniq=dfComptagesExistants.id)
                                                                         , ['id_comptag_uniq', 'pc_pl']), {'valeur':'pc_pl'}, identifiant='id_comptag_uniq',
                filtre = "indicateur='pc_pl'")

#insertion des nouveaux
for a in dfComptagesNew.annee.unique() : 
    dfComptage=cd64.creer_comptage(dfComptagesNew.loc[dfComptagesNew.annee==a].id_comptag.tolist(), a, 'tableur CD64','tv/pl')
    cd64.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dfComptage)
    dfIndicAgrege=cd64.structureBddOld2NewForm(dfComptagesNew.loc[dfComptagesNew.annee==a], a,
                                               ['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege')
    cd64.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege', dfIndicAgrege)

# ***CD47***
> Annee des trafics 2018, fichiers : Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD47\comptages_CD47_2018\COMPTAGES 2018
les fichiers sont decomposes en permanents tournants temporaires, il faut recomposer les donnees

In [ ]:
#initialiser un objet
cpt47=it.Comptage_cd47(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD47\comptages_CD47_2018\COMPTAGES 2018','TRAFICS PERMANENTS')
#calculer les attributs de comptages : df_attr, df_attr_insert et df_attr_update, en prenant en compte les comptages existants
cpt47.classer_comptage_update_insert('gti_otv_pg11')

In [ ]:
#mise à jour des données déjà présentes dansla base
val_txt=cpt47.creer_valeur_txt_update(cpt47.df_attr_update, ['id_comptag','tmja','pc_pl'])
cpt47.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja','pc_pl_2018':'pc_pl'})

In [ ]:
#traitement des données non présentes (df_attr_insert)
#recherche de correspondance pour les permanents et tournants
dico_corresp=cpt47.corresp_old_new_comptag('gti_otv_pg11', 'public','cpt47_temp','lineaire.traf2017_bdt47_ed17_l',
                            'referentiel', 'troncon_route_bdt47_ed17_l','troncon_route_bdt47_ed17_l_vertices_pgr','id')

In [ ]:
#filtre des données df_attr_insert selon l'id_comptag present dans le dico_corresp
cpt47.df_attr_insert=cpt47.df_attr_insert.loc[~cpt47.df_attr_insert.id_comptag.isin(df_correspondance.id_comptag.to_list())].copy()
# POUR INFO, l'id_comptag présent dan sle dico_corresp a été transférer à la main dans l table comptage.corresp_id_comptag
#mettre en forme les attributs avant insert
cpt47.mise_en_forme_insert('2018')
#inserer les donnes dans la table
cpt47.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2018_p', cpt47.df_attr_insert)

In [ ]:
#mettre à jour la géométrie
cpt47.maj_geom('gti_otv_pg11', 'comptage', 'na_2010_2018_p', '47')

> ## **2019**

In [ ]:
cpt47=it.Comptage_cd47(r'D:\temp\otv\2019\Donnees_source\CD47','TRAFICS PERIODIQUES',2019)
#calculer les attributs de comptages : df_attr, df_attr_insert et df_attr_update, en prenant en compte les comptages existants, mais ans recherche des équivalences avec les anciens comptage snon recensées
cpt47.classer_comptage_update_insert('local_otv_station_gti')

In [ ]:
#qq points tournants non connus
cpt47.df_attr_insert.loc[cpt47.df_attr_insert['type_poste'].isin(['permanent','tournant'])]
#filtrer les 2 points bizarres
cpt47.df_attr=cpt47.df_attr.loc[cpt47.df_attr.id_comptag.apply(lambda x : x[:4]=='47-D')].copy()
cpt47.df_attr_insert=cpt47.df_attr_insert.loc[cpt47.df_attr_insert.id_comptag.apply(lambda x : x[:4]=='47-D')].copy()
cpt47.df_attr_update=cpt47.df_attr_update.loc[cpt47.df_attr_update.id_comptag.apply(lambda x : x[:4]=='47-D')].copy()

In [ ]:
#obtention dico de correspondance
dico_corresp=cpt47.corresp_old_new_comptag('local_otv_station_gti', 'public','cpt47_temp','referentiel.troncon_route_bdt47_ed17_l',
                            'referentiel', 'troncon_route_bdt47_ed17_l','troncon_route_bdt47_ed17_l_vertices_pgr','id')
#insertion dans bdd
cpt47.insert_bdd('local_otv_station_gti','comptage','corresp_id_comptag',dico_corresp[['id_comptag','id_comptag_lin']].rename(columns={'id_comptag':'id_gest','id_comptag_lin':'id_gti'}))
#calcul nouveau id_comptag
cpt47.comptag_existant_bdd('local_otv_station_gti',schema='comptage', table='na_2010_2019_p',dep='47', type_poste=False)
cpt47.corresp_nom_id_comptag('local_otv_station_gti',cpt47.df_attr)
cpt47.df_attr_update=cpt47.df_attr.loc[cpt47.df_attr.id_comptag.isin(cpt47.existant.id_comptag.tolist())].copy()
cpt47.df_attr_insert=cpt47.df_attr.loc[~cpt47.df_attr.id_comptag.isin(cpt47.existant.id_comptag.tolist())].copy()

In [ ]:
#mise a jour
cpt47.update_bdd_47('local_otv_station_gti','comptage', 'na_2010_2019_p',cpt47.df_attr_update)

In [ ]:
#donnees a inserer
cpt47.mise_en_forme_insert()
cpt47.insert_bdd('local_otv_station_gti','comptage', 'na_2010_2019_p', cpt47.df_attr_insert)
cpt47.maj_geom('local_otv_station_gti','comptage', 'na_2010_2019_p', '47')
cpt47.insert_bdd('local_otv_station_gti','comptage', 'na_2010_2019_mensuel', cpt47.df_attr_mens)
cpt47.insert_bdd('local_otv_station_gti','comptage', 'na_2010_2019_horaire', cpt47.df_attr_horaire)

# ***CD87***
> Dans ce Departement les donnees sont fournies en fichiers .fim. il vaut recalculer les valeusr de comptages <br> Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD87\Fichiers FIM 87 -2018

### ***lister les voies, classer les types de commptages, lister les fichiers a regrouper***
> le dossier contient un tres grand nombre de fihciers, parfois ils sont a regrouper, parfois la structure de nommage varie

In [ ]:
d87=it.Comptage_cd87(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD87\CD87- Comptages 2019 Fichiers .fim','2019') 
d87.dico_pt_cptg() #creer le dico de base

In [ ]:
d87.dico_voie

### ***Traiter les donnees***
> Pour chaque route référencée dans le dico, on va calculer les TMJA %PL, date_debut, date_fin, type_poste de chaque fichiers puis
faire les calculs si necessaires (moyenne si plsr fichiers).<br> ensuite on classe puis update et insert

In [ ]:
#mettre à jour le dico et le transformer en dataframe sans les ponctuels pendant les grandes vacances
d87.dataframe_dico_glob()

In [ ]:
#preparer les données
#attention cela comprend un script d'identification des points pas forcément nécéssaire
d87.classer_comptage_update_insert('gti_otv_pg11','na_2010_2019_p','comptage',
                                   'public','d87_cpt_temp','lineaire.traf2018_bdt87_ed18_l',
                                  'ref', 'troncon_route_bdt87_ed18_l','troncon_route_bdt87_ed18_l_vertices_pgr','id','ref.pr_ed18_p')

In [ ]:
#mettre à jour les données
d87.update_bdd_d87('gti_otv_pg11','na_2010_2019_p','comptage')
#insérer les données et mettre à jour les geom
d87.insert_bdd_d87('gti_otv_pg11','na_2010_2019_p','comptage')

In [318]:
Outils.CopierFichierDepuisArborescence(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD87\Permanents 2019\2019',
                                      r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD87\Permanents 2019\tousFichiersVrac',
                                      '.fim')

In [338]:
d87=it.Comptage_cd87(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD87\Permanents 2019\tousFichiersVrac','2019') 
d87.dico_pt_cptg() #creer le dico de base

OK : tous fichiers dans dico


In [343]:
d87.df_attr.sort_values('id_comptag')

,route,pr,absc,tmja,pc_pl,type_poste,date_debut,date_fin,id_comptag
4,D2000,15,105,9178,8.25,permanent,NaN,NaN,87-D2000-15+105
3,D2000,22,250,5182,10.53,permanent,NaN,NaN,87-D2000-22+250
5,D29,4,830,8372,2.89,permanent,NaN,NaN,87-D29-4+830
6,D675,26,970,2942,7.37,permanent,NaN,NaN,87-D675-26+970
8,D675,31,0,2025,6.76,permanent,NaN,NaN,87-D675-31+0
7,D675,53,890,2786,4.02,permanent,NaN,NaN,87-D675-53+890
9,D675,58,680,10367,4.08,permanent,NaN,NaN,87-D675-58+680
1,D699,2,255,1941,4.93,permanent,NaN,NaN,87-D699-2+255
13,D704,1,495,6386,5.61,permanent,NaN,NaN,87-D704-1+495
14,D704,19,720,4648,4.82,permanent,NaN,NaN,87-D704-19+720


# ***CD 16***
- Année 2018 : 
> à partir du fichier D:\temp\otv\Donnees_source\CD16\B15_2018.xlsx on traite les permanents.<br> à partir des donnees sur PIGMA on traite les compteurs temporaires <br>***attention : il y a aussi des données de comptages temporaires en FIM qui permettront d'obtenir de la données horaires***

In [ ]:
#2018
cd16=it.Comptage_cd16(r'D:\temp\otv\Donnees_source\CD16\B15_2018.xlsx',r'D:\temp\otv\Donnees_source\CD16\pigma\comptages_routiers_2019\comptages_routiers.shp',
                      r'D:\temp\otv\Donnees_source\CD16\pigma\position_compteurs_2019\position_compteurs.shp',2018)
cd16.comptage_forme()
cd16.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')

cd16.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')

cd16.update_bdd_16('local_otv','comptage','na_2010_2018_p')

In [ ]:
#2019
cd16=it.Comptage_cd16(r'D:\temp\otv\2019\Donnees_source\CD16\B15_2019.xlsx',r'D:\temp\otv\2019\Donnees_source\CD16\SIG_Comptages_CD16\Comptages_routiers.shp',
                      r'D:\temp\otv\2019\Donnees_source\CD16\SIG_Comptages_CD16\Position_compteurs.shp',2019)
cpt_a_ignorer=('16-D731-27+45','16-D910-23+821', '16-D1000-16+35')
cd16.comptage_forme(7,('16-D731-27+45','16-D910-23+821', '16-D1000-16+35'),r'D:\temp\otv\2019\Donnees_source\CD16\Comptages_secondaires_CD16_2019')
cd16.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2019_p')
#cd16.update_bdd_16('local_otv_station_gti','comptage', 'na_2010_2019_p')
#cd16.insert_bdd('local_otv_station_gti','comptage','na_2010_2019_mensuel',cd16.df_attr_mens)
#cd16.insert_bdd('local_otv_station_gti','comptage','na_2010_2019_horaire',cd16.df_attr_horaire)

# ***CD 86***
Attention , dans ce département parfois ce sont les données en PLOD et ASCD qui sont ok, parfois c'est le cumulD <br>
Dans ce Dept, les Compmtages dites 'Secondaires' sont équivalent à 'tournants', et 'Tournants' equivalent à 'ponctuel'
- Année 2018 : 
> à partir des fichier D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx et  D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls on traite les permanents et secondaires <br> il y a un petit pb sur les compteurs permanents entre la donnees pr+abs chez nous et la leur dans le tableau, donc il faut la premiere fois tout passer dans la table de correspondance.<br> dans ce dept pas de nouveau points
- Année 2019 : 
> un seul fichier avec les comptages sur 2feuilles : permanents et tournants.<br> la feuille permanent contient aussi des données 'Tournant' qui sont masquées, pour lesquelles les reference en abscisse sont tuojours à 0<br> 3 nouveaux points tournants ajoutés à  la main, le reste des ponctuels en auto

In [ ]:
#2018
cd86=it.Comptage_cd86(r'D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx',r'D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls')

#si besoin de dico corresp : 
corr=cd86.corresp_perm('local_otv', 'na_2010_2018_p')
cd86.insert_bdd('local_otv', 'comptage','corresp_id_comptag',corr)

#sinon
cd86.comptage_forme()
cd86.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')
cd86.update_bdd_86('local_otv','comptage','na_2010_2018_p')

In [ ]:
#2019
cd86=it.Comptage_cd86(r'D:\temp\otv\2019\Donnees_source\CD86\Comptages globaux 2019 tournants et permanents.xlsx',r'D:\temp\otv\2019\Donnees_source\CD86\Comptages globaux 2019 tournants et permanents.xlsx',2019,
                     'permanents','secondaires')
cd86.comptage_forme('local_otv_station_gti', 'na_2010_2019_p')
cd86.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2019_p')
cd86.update_bdd_86('local_otv_station_gti','comptage','na_2010_2019_p')

In [ ]:
cd86.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2019_p')

In [ ]:
cd86.df_attr_insert['dep']='86'
cd86.df_attr_insert['reseau']='RD'
cd86.df_attr_insert['gestionnai']='CD86'
cd86.df_attr_insert['concession']='N'
cd86.df_attr_insert['obs']="nouveau_point 2019, denomination CD86='tournant'"
cd86.df_attr_insert.rename(columns={'absc' : 'abs', 'tmja':'tmja_'+str(cd86.annee),'pc_pl':'pc_pl_'+str(cd86.annee),'obs':'obs_'+str(cd86.annee), 'src':'src_'+str(cd86.annee)},inplace=True)

In [ ]:
cd86.insert_bdd_86('local_otv_station_gti', 'comptage','na_2010_2019_p')

In [5]:
with ct.ConnexionBdd('local_otv_gti', localisation='maison') as c : 
    ct.ogr2ogr_csv2pg(c.connstringOgr, r'F:\Boulot\otv\corresp_id_comptag.csv',schema='public', table='corr_id',encodageClient='UTF-8', headers='YES')

debut import fichier F:\Boulot\otv\corresp_id_comptag.csv avec shape2pg à 09:18:21 
 avec commande ogr2ogr -f "postgreSQL" --config PG_USE_COPY YES -lco "SCHEMA=public" PG:"host=localhost dbname=otv_gti user=postgres password=postgres port=5432" F:\Boulot\otv\corresp_id_comptag.csv -nln public.corr_id -oo HEADERS=YES
Fait


# ***CD 24***
- Année 2018 : 
> à partir des fichier D:\temp\otv\Donnees_source\CD24\2018_CD24_trafic.csv <br> il n'y a que les compteurs permanents.<br> **Données Mensuelles dispos**.<br> Pensez que des céhanges ont ue lieu pour récupérer tous les points de comptages (perm, tourn, ponct). <br> ***Gros pb de geooloc des PR ign***

In [ ]:
cd24=it.Comptage_cd24(r'D:\temp\otv\Donnees_source\CD24\2018_CD24_trafic.csv')
cd24.comptage_forme()
cd24.classer_comptage_update_insert('local_otv_station_gti', 'na_2010_2018_p')

In [ ]:
cd24.update_bdd_24('local_otv_station_gti', 'comptage', 'na_2010_2018_p')

In [ ]:
cd24.insert_bdd_24('local_otv_station_gti', 'comptage', 'na_2010_2018_p')

Année 2019 : 
>à partir du fichier Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD24
que les compteurs permanents et tournants.
a voir pour les ponctuels, possibilité aussi de récupérer des données à partirde leur Web SIG : https://dordogne.maps.arcgis.com/apps/MapTools/index.html?appid=34558f68af514a63b6b7426ed77d055f en scrolant et enregistrant les différents fichiers html. ensuite pandas a une fonction read_html qu el'on applique sur des slices du fichiers html (car la fonction renvoi tout les elements dans des slices diffrentes : ligne du tableau, carte, etc...)<br> IL POURRAIT ETRE INTERESSANT DE METTRE A JOUR LES DONNEES DE GEOM SI BESOIN AVCE LES LONG/LAT issues du fichier source du CD24 quand c'est possible

In [50]:
cd24=it.Comptage_cd24(r'D:\Boulot\AffairesEnCours\OTV\24_donnees_sources\EXPORT SIG COMPTAGES CD24.csv',2019)
cd24.comptage_forme()
cd24.classer_comptage_update_insert('local_otv_gti', 'na_2010_2019_p')

c:\users\marti\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:830: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [21]:
cd24.update_bdd_24('local_otv_gti', 'comptage', 'na_2010_2019_p')
cd24.insert_bdd('local_otv_gti', 'comptage', 'na_2010_2019_p', cd24.df_attr_insert)
cd24.insert_bdd('local_otv_gti', 'comptage', 'na_2010_2019_mensuel', cd24.df_attr_mens)

c:\users\marti\appdata\local\programs\python\python38\lib\site-packages\geopandas\geodataframe.py:830: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


# ***CD 33***
- Année 2019 : 
> à partir du fichier de comptage C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\OTV\33\CD33\export  2019 pour DREAL.xlsx (aussi présent sur les anciens serveurs) et du fichier de sectionnement C:\Users\martin.schoreisz\Box\Dossier_Personnel_de_Martin_SCHOREISZ\OTV\33\CD33\SECT_2021_CAT en cours.shp <br> On va chercher à recouper les comptages avec les notres en associant l'identifiant de troncon.<br> les troncons du CD ont l'air cohérents, on va qd mm checker les regrouepemnt <br> **Données Mensuelles dispos**<br> environ 70 cpt permanents et 60 tournants

In [102]:
cptPerm33=it.Comptage_cd33( r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD33\export_2019_pour_DREAL.xlsx',2019,
                         r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\CD33\SECT_2021_CAT en cours.shp')

In [159]:
#trouver les connus et inconnus
GdfPerm,gdfPermConnus, gdfPermInconnus=cptPerm33.trierPermConnus('local_otv_boulot', 'na_2010_2019_p', 'boulot')
#assigner les inconnu manuellement
dico_corresp={'0106_03':'33-D106-35+835'}
cptPerm33.assignerCptInconnus(dico_corresp,gdfPermInconnus)
#fusion
gdfPermConnus=pd.concat([gdfPermConnus, gdfPermInconnus.loc[~gdfPermInconnus.id_comptag.isna()]])
gdfPermInconnus=gdfPermInconnus.loc[gdfPermInconnus.id_comptag.isna()]
GdfPerm=pd.concat([gdfPermConnus,gdfPermInconnus])
#mensuel
donnees_mens_perm=GdfPerm[['troncon']+[m for m in it.dico_mois.keys()]].assign(donnees_type='tmja')

In [182]:
#comptages tournants
tournant=cptPerm33.analyseTourn()
donneesMensTour=cptPerm33.donneesMensTournant(tournant)

In [ ]:
#chercher les correspondance
rqtPpvCptTournant="""SELECT DISTINCT ON (c.troncon) t.id_ign, c.troncon
 FROM test_affectation_cd33_tournant t JOIN tournant_geoloc_l93 c ON st_dwithin(c.geom, t.geom,30)
 ORDER BY c.troncon, st_distance(t.geom, c.geom)"""
rqtCptExistant="select id_ign, id_comptag from lineaire.traf2019_bdt33_ed19_l"
rqtGeomReferentielEpure='select id_ign, geom from test_affectation_cd33_tournant'
rqtPpvCptBdd="""SELECT DISTINCT ON (t.id_ign) t.id_ign, c.id_comptag
 FROM test_affectation_cd33_tournant t JOIN comptage.na_2010_2019_p c ON st_dwithin(c.geom, t.geom,30)
 WHERE c.gestionnai='CD33'
 ORDER BY t.id_ign, st_distance(t.geom, c.geom)"""
correspTournant,inconnuTournant=cptPerm33.correspondanceTournant('gti_otv_pg11', 'boulot', rqtPpvCptTournant, rqtCptExistant, rqtGeomReferentielEpure, rqtPpvCptBdd,
                               'public', 'test_affectation_cd33_tournant','test_affectation_cd33_tournant_vertices_pgr', tournant )

In [109]:
#apres analyse mano : 
dicoCorrespTourn={'0005_06':'33-D5-76+0',
                  '0012_11':'33-D12-17+200',
                  '0115_06' : '33-D115-103+0',
                 '0116_01':'33-D116-23+0',
                 '0216_02':'33-D216-15+0',
                 '0222_02':'33-D222-16+0',
                 '0655_01':'33-D655-10+0'}
cptTournantAffecte=cptPerm33.correctionCorrespondanceTournant(dicoCorrespTourn,tournant,correspTournant)

In [113]:
#creer les df_attr sur le mm schema que les habituels
cptPerm33.df_attr_update=pd.concat([cptTournantAffecte.loc[cptTournantAffecte.correspondance==True][['id_comptag', 'tmja_2019', 'pc_pl_2019', 'obs_2019', 'src_2019', 'fichier']],
gdfPermConnus[['id_comptag', 'tmja_2019', 'pc_pl_2019', 'src_2019', 'fichier']]])

In [144]:
#creer nouveaux points tournants
dicoNewCpt={'troncon':['0652_00','0003_17','0005_00','0009_02','0011_05','0012_10','0012_13','0110_01','0214_00','0222_01','0651_02','0932_03'],
             'pr':[4,163,61,32,55,11,39,15,3,5,34,0],
             'absc':[280,60,600,90,760,0,330,885,980,280,20,812]}
cptPerm33.df_attr_insert=cptPerm33.creerNouveauPointTournants(cptTournantAffecte,dicoNewCpt)

In [135]:
cptPerm33.df_attr_insert
cptPerm33.df_attr_update

In [184]:
#donnees mensuelles
donnees_mens_perm.merge(GdfPerm[['troncon', 'id_comptag']], on='troncon')
df_corresp_mens=cptTournantAffecte[['troncon','id_comptag']].merge(cptPerm33.df_attr_insert[['troncon','id_comptag']], on='troncon', how='left')
df_corresp_mens['id_comptag']=df_corresp_mens.apply(lambda x : x['id_comptag_x'] if not pd.isnull(x['id_comptag_x']) else x['id_comptag_y'], axis=1)
df_attr_mens=pd.concat([donneesMensTour.merge(df_corresp_mens[['troncon', 'id_comptag']], on='troncon'),
                        donnees_mens_perm.merge(GdfPerm[['troncon', 'id_comptag']], on='troncon')]).drop('troncon', axis=1)

In [186]:
cptPerm33.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2019_mensuel',df_attr_mens.assign(annee=str(cptPerm33.annee)))

# ***SCA***
- Année 2018 : 
> Concerne COFIROUTE, VINCI, ALIENOR, ATLANDES.<br> Des données de correspondances id_comptages sont dans la table source de la base de données otv.<br> **Donnees TMJM dispos**

- ### COFIROUTE 
> Que 5 points, faits à la main

<a id="6" ></a>
- ### VINCI 
> L'idée c'est d'importer le tableur et les données et de faire la jointure pour mise à jour.<br>**ATTENTION : rien de prévu si nouveau points** 

In [ ]:
#2018
cpt=it.Comptage_vinci(r'D:\temp\otv\Donnees_source\VINCI\2018_comptage_vitesse_moyenne_VINCI.xlsx')
cpt.update_bdd_Vinci('local_otv_station_gti', 'comptage', 'na_2010_2018_p')

In [ ]:
#2019
cpt=it.Comptage_vinci(r'D:\temp\otv\2019\Donnees_source\ASF\bdd_point_comptage et vitesse moyenne 2019.xlsx', 2019)
cpt.update_bdd_Vinci('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
#cpt.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',cpt.df_attr_mens)

In [5]:
#2020 : pb de jointure entre les donnees et les id_comptag en base, donc je fais simple et je copie d'id comptage manuellement
cpt=it.Comptage_vinci(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\ASF\en_cours\bdd_point_comptage et vitesse moyenne 2020.xlsx', 2020)
cpt.classer_comptage_update_insert('local_otv_boulot', 'compteur')
#cpt.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',cpt.df_attr_mens)

In [40]:
#insertion comptage
cpt.insert_bdd('local_otv_boulot', 'comptage', 'comptage',cpt.creer_comptage(cpt.df_attr.id_comptag.tolist(), cpt.annee, 'tableur Vinci', 'tv/pl'))

In [46]:
#insertion agrege
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',
               cpt.structureBddOld2NewForm(cpt.df_attr_update, cpt.annee,['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))

In [48]:
#insertion mensuel
cpt.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',cpt.structureBddOld2NewForm(cpt.df_attr_mens, 
            cpt.annee,['id_comptag', 'annee', 'fichier', 'donnees_type'],['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept',
       'octo', 'nove', 'dece'], 'mensuel'))

<a id="4" ></a>
- ### ALIENOR / ATLANDES
> Que 7 points chaucun, j'utilise surtout là pour les donees mensuelles

In [ ]:
alienor=it.Comptage_alienor(r'D:\temp\otv\2019\Donnees_source\ALIENOR\ALIENOR_trafic_2019.xlsx',2019)
alienor.ouvrir_et_separe_donnees()
alienor.update_bdd_Alienor('local_otv_station_gti', 'comptage', 'na_2010_2019_p')
alienor.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',alienor.df_attr_mens)

In [ ]:
atlandes=it.Comptage_atlandes(r'D:\temp\otv\2019\Donnees_source\ATLANDES\2019_trafic_atlandes_7_boucles.xls',2019)
atlandes.donnees_mens()
atlandes.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_mensuel',atlandes.df_attr_mens)

In [349]:
#2020
atlandes=it.Comptage_atlandes(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\ATLANDES\en_cours\Trafic atlandes 2020 - 7 boucles DREAL.xls', '2020')
donnees=atlandes.miseEnForme()
atlandes.insert_bdd('local_otv_boulot', 'comptage', 'comptage',atlandes.donneesAgregees()[['id_comptag', 'src', 'type_veh', 'annee']] )
atlandes.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',
                    atlandes.structureBddOld2NewForm(atlandes.donneesAgregees(),atlandes.annee, ['id_comptag', 'annee', 'fichier'], ['tmja', 'pc_pl'], 'agrege'))
atlandes.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',
                    atlandes.structureBddOld2NewForm(atlandes.df_attr_mens, atlandes.annee,['id_comptag', 'donnees_type', 'annee', 'fichier'], ['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil',
       'aout', 'sept', 'octo', 'nove', 'dece'], 'mensuel' ))

# ***Grand Poitiers***
> on a deja un fichier dans la base de donnees : on va chercher a greffer les donnees et a cree un nouvel id_comptag

In [ ]:
#ouvrir un fichier
dossier=r'D:\temp\otv\2019\Donnees_source\GP\Automatiques 2019'
cpt=it.Comptage_GrandPoitiers(r'D:\temp\otv\2019\Donnees_source\GP\Automatiques 2019',2019)
cpt.comptage_forme('local_otv_station_gti')
cpt.update_bdd_grdPoi('local_otv_station_gti', 'comptage', 'na_2010_2019_p')

In [ ]:
cpt.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_horaire', cpt.df_attr_horaire)

<a id=10 ></a>
# ***Ville Anglet***
> Ci-dessous simplement un exemple d'ouverture de fcihier .mdb, de liste des tables et de lecture d'une table

In [3]:
Anglet=it.Comptage_Anglet('2017',r'C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet\COMPTAGES ANGLET\Comptages 2017\données brutes',
                         r'C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet')

In [4]:
#epxortdu fichier csv seravnt a la geoloc
Anglet.exporterCsvAGeocoder()
#enuiste on geoloc a la main, dans le repertoir dossierResume

In [5]:
#puis on recupere la geoloc et on creer la df permettantde creer des compteurs
Anglet.creerDfGeoloc()

In [ ]:
#creation des df hoarires et agrege sans filtre
Anglet.indicsTousFichiers()

In [9]:
dfPPV=Anglet.plusProcheVoisinBddRegroupe('traf2020_bdt_na_ed20_simpli_l')

In [114]:
#trouver les doublons
listCptDoublons=Anglet.isolerComptagesDoublons(dfPPV)
#apres verif on conserve
cptAConserver=['Anglet-10_rue_de_Chassin--1.5343;43.4819', 'Anglet-53_rue_de_Chassin--1.5323;43.4848', 'Anglet-103_Av_de_l_Adour--1.5002;43.5127', 
'Anglet-20_rue_de_Lavigne--1.5267;43.4724','Anglet-277_Av_de_l_Adour--1.5118;43.5262','Anglet-31_Av_de_la_Ch_d_Amour--1.5349;43.4952','Anglet-Av_Chambre_d_Amour(petit_casino)--1.5317;43.4949']
#filtre
dfPPVSsDbl=Anglet.cptSsDblEtSsGeom(dfPPV, cptAConserver)

In [114]:
#inserer donnees 
Anglet.

In [70]:
#creer les compteurs a inserer dans la bdd
dfCptAcreer=dfPPVSsDbl.loc[dfPPVSsDbl.id_comptag_bdd.isna()]
compteur=dfCptAcreer[['geometrie', 'id_comptag', 'route', 'reseau', 'dep', 'gestionnai','concession', 'type_poste', 'src_geo', 'obs_geo', 'x_l93', 'y_l93',
       'fictif', 'src_cpt', 'convention','obs_supl','sens_cpt']].drop_duplicates().copy()
#inserer
Anglet.insert_bdd('local_otv_boulot', 'comptage', 'compteur', compteur)

In [99]:
#creer les comptages a inserer dans la bdd
dfPPVSsDbl['id_comptag_final']=dfPPVSsDbl.apply(lambda x : x.id_comptag_bdd if not pd.isnull(x.id_comptag_bdd) else x.id_comptag, axis=1)
comptage=dfPPVSsDbl[['id_comptag_final', 'annee', 'periode']].assign(src='gestionnaire', type_veh='tv/pl').rename(columns={'id_comptag_final':'id_comptag'}).drop_duplicates()
#inserer
Anglet.insert_bdd('local_otv_boulot', 'comptage', 'comptage',comptage)

In [172]:
#creer les indicateurs a inserer dans la Bdd
indicAgrege=dfPPVSsDbl.merge(Anglet.recupererIdUniqComptage(dfPPVSsDbl.id_comptag_final.tolist(), Anglet.annee).rename(columns={'id_comptag_final':'id_comptag'})
                             , on=['id_comptag', 'annee'])[['id_comptag_uniq', 'indicateur', 'valeur', 'fichier']]
dfHoraireSsDbl=Anglet.dfHoraire.merge(dfPPVSsDbl[['id_comptag', 'id_comptag_final']], on='id_comptag')
indicHoraire=dfHoraireSsDbl.assign(annee=Anglet.annee, 
    indicateur=dfHoraireSsDbl.indicateur.str.upper()).merge(Anglet.recupererIdUniqComptage(
    dfHoraireSsDbl.id_comptag_final.tolist(), Anglet.annee).rename(columns={'id_comptag':'id_comptag_final'}), on=['id_comptag_final', 'annee']).drop([
    'sens_cpt', 'note_manuelle_qualite', 'obs_qualite', 'annee', 'id_comptag','id_comptag_final', 'obs_supl'], axis=1)
Anglet.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',indicAgrege)
Anglet.insert_bdd('local_otv_boulot', 'comptage', 'indic_horaire',indicHoraire)

In [168]:
len(indicHoraire.id_comptag_uniq.unique())

44

In [170]:
indicHoraire.loc[indicHoraire.id_comptag_uniq.isna()]

,jour,indicateur,h0_1,h1_2,h2_3,h3_4,h4_5,h5_6,h6_7,h7_8,h8_9,h9_10,h10_11,h11_12,h12_13,h13_14,h14_15,h15_16,h16_17,h17_18,h18_19,h19_20,h20_21,h21_22,h22_23,h23_24,fichier,id_comptag_uniq


In [3]:
#paramétrer les infos relatives a une annee
annee='2018'#2019
dossier=r'C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet\COMPTAGES ANGLET\Comptages 2018\données brutes'#C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet\COMPTAGES ANGLET\Comptages 2019\données brutes
dossierResume=r'C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet'
codepostal='64600'
Commune='Anglet'
resumeComptage=os.path.join(dossierResume,f'Comptage{annee}.csv')
resumeComptageGeoloc=os.path.join(dossierResume,f'Comptage{annee}_geoloc.csv')

In [247]:
#lister les fichiers et exporter le fichier csv pour géocodage IGN
dicoAdresse={'fichier':[], 'adresse':[], 'codePostal':codepostal, 'Commune':Commune}
#exporter les adresses (géocodées ensuite via mongeocodeur)
def replaceAdresse(matchobj):
    return matchobj.group(0).replace('.',',')
for files in os.listdir(dossier):
    if os.path.isfile(os.path.join(dossier, files)) and files.endswith('.mdb'):
        dicoAdresse['fichier'].append(os.path.join(dossier, files))
        dicoAdresse['adresse'].append(re.sub('^[1-9]+\.', replaceAdresse,files[:-4] ))
dfFichierAdresse=pd.DataFrame(dicoAdresse)
dfFichierAdresse.to_csv(resumeComptage, sep=';')

In [328]:
#importer le fichier des points de comptage geolocalise et le mettre en forme
gdfGeoloc=gp.read_file(resumeComptageGeoloc, encoding='utf-8')#C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet\Comptage2019_geoloc.csv
gdfGeoloc.geometry=gp.points_from_xy(x=gdfGeoloc['longitude'], y=gdfGeoloc['latitude'], crs=4326)
gdfGeoloc=gdfGeoloc.to_crs('epsg:2154')
gdfGeoloc['id_comptag']=gdfGeoloc.apply(lambda x : 'Anglet-'+re.sub('_{2,50}', '_',re.sub('(,|\.|\'| +)', '_', x['adresse']))+f"-{round(float(x['longitude']), 4)};{round(float(x['latitude']),4)}", axis=1)
gdfGeoloc['route']=gdfGeoloc.adresse.apply(lambda x : re.sub('^([1-9]{1,3},)+','', x))
gdfGeoloc['reseau']='VC'
gdfGeoloc['dep']='64'
gdfGeoloc['gestionnai']='Anglet'
gdfGeoloc['concession']=False
gdfGeoloc['type_poste']='ponctuel'
gdfGeoloc['src_geo']='adresse'
gdfGeoloc['obs_geo']=gdfGeoloc.apply(
    lambda x : f"geocodé avec 'mon géocodeur' de l'IGN ; qualite : {x.qualite} ; precision geocodage : {x['precision geocodage']} ; ID adresse : {x['ID adresse']}; adresse geocodee : {x['adresse geocodee']}", axis=1)
gdfGeoloc['x_l93']=round(gdfGeoloc.geometry.x,3)
gdfGeoloc['y_l93']=round(gdfGeoloc.geometry.y,3)
gdfGeoloc['fictif']=False
gdfGeoloc['src_cpt']='gestionnaire'
gdfGeoloc['convention']=False

In [ ]:
#récupérer un fichier depuis le mdb et en extraire les donnees horaires
listDfHoraires=[]
dicoNbSens={}
for row in gdfGeoloc.itertuples(index=False) : 
    print(row.fichier)
    mhc=MHCorbin(row.fichier)
    listDfHoraires.append(mhc.formaterDonneesHoraires(mhc.formaterDonneesIndiv(mhc.dfAgreg2Sens)).reset_index().assign(
        id_comptag=row.id_comptag,
        fichier=mhc.fichierCourt,
        obs_supl=f"{mhc.dfAgreg2Sens.obs_supl.values[0]} ; {mhc.dfAgreg2Sens.sensTxt.values[0]}",
        sens_cpt=mhc.dfAgreg2Sens.nb_sens.values[0],
        note_manuelle_qualite=mhc.indicQualite,
        obs_qualite=mhc.comQualite))
#dfHoraireBddAnglet=Anglet.formaterDonneesHoraires(Anglet.formaterDonneesIndiv(Anglet.dfAgreg2Sens))

In [331]:
#calcul des dF d'indicateurs
dfHoraire=pd.concat(listDfHoraires)
dfIndicAgrege=tmjaDepuisHoraire(dfHoraire.assign(annee=annee)).merge(dfHoraire[['id_comptag','obs_supl','sens_cpt', 'note_manuelle_qualite', 'obs_qualite', 'fichier']].drop_duplicates(), on='id_comptag', how='left')

In [332]:
#passer la df dans bdd pour plus propche (cas du premier jeu de donnees passe) 
transfert=it.Comptage('fake')
transfert.insert_bdd('local_otv_boulot', 'public', f'cpt_anglet_{annee}',gdfGeoloc.drop('fichier', axis=1), if_exists='replace' )

c:\users\martin.schoreisz\appdata\local\programs\python\python39\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [297]:
#trouver le plus proche voisin sur les tronçons elementaires de Vince (DF initiale)
rqt="""SELECT DISTINCT ON (c.field_1) c.*, t.gid, st_distance(c.geom, t.geom)
 FROM public.cpt_anglet_2019_temp c left JOIN linauto.traf2020_bdt_na_ed20_simpli_l t ON st_dwithin(c.geom, t.geom, 20)
 ORDER BY c.field_1, st_distance(c.geom, t.geom)"""
with ct.ConnexionBdd('local_otv_boulot') as c : 
    dfPPV=gp.read_postgis(rqt, c.sqlAlchemyConn, crs='epsg:2154').merge(dfIndicAgrege, on='id_comptag', how='left').merge(periodeDepuisHoraire(dfHoraire.assign(annee='2019')), how='left', on='id_comptag')
    dfPPV=O.gp_changer_nom_geom(dfPPV, 'geometrie')

In [333]:
#trouver le plus proche voisin sur les tronçons elementaires de Vince avec en plus une colonne qui traduit si le point existe deja
rqt=f"""WITH 
cpteur_base AS (
SELECT DISTINCT ON (c.id_comptag) c.id_comptag id_comptag_bdd, t.gid, st_distance(c.geom, t.geom)
 FROM comptage.compteur c left JOIN linauto.traf2020_bdt_na_ed20_simpli_l t ON st_dwithin(c.geom, t.geom, 20)
 ORDER BY c.id_comptag, st_distance(c.geom, t.geom))
SELECT DISTINCT ON (c.field_1) 
                    c.*, t.gid, st_distance(c.geom, t.geom),
                    c2.id_comptag_bdd
 FROM public.cpt_anglet_{annee} c left JOIN linauto.traf2020_bdt_na_ed20_simpli_l t ON st_dwithin(c.geom, t.geom, 20)
                               LEFT JOIN cpteur_base c2 ON t.gid=c2.gid
 ORDER BY c.field_1, st_distance(c.geom, t.geom)"""
with ct.ConnexionBdd('local_otv_boulot') as c : 
    dfPPV=gp.read_postgis(rqt, c.sqlAlchemyConn, crs='epsg:2154').merge(dfIndicAgrege, on='id_comptag', how='left').merge(periodeDepuisHoraire(dfHoraire.assign(annee=annee)), how='left', on='id_comptag')
    dfPPV=O.gp_changer_nom_geom(dfPPV, 'geometrie')

In [198]:
#pour les comptages qui sont multiples et proches d'une voies, je ne garde que le plus fort (DF initiale)
dfPPVTmja=dfPPV.loc[(dfPPV.indicateur=='tmja') & (~dfPPV.gid.isna())]
dfCptUniq=dfPPV.loc[dfPPV.id_comptag.isin(dfPPVTmja.loc[dfPPVTmja.valeur==dfPPVTmja.groupby('gid').valeur.transform(max)].id_comptag.unique())]
Comptag_2019=pd.concat([dfCptUniq, dfPPV.loc[dfPPV.gid.isna()]])

In [370]:
#conserver certains comptages, mais en priorisant les plus fort quand même
#compteurs a creer 
idComptagForce=('Anglet-12_bis_Av_de_la_Chambre_d_Amour--1.5306;43.495', 'Anglet-16_Av_de_la_Chambre_d_Amour--1.5307;43.4949')
dfPPVForce=dfPPV.loc[(dfPPV.id_comptag.isin(idComptagForce)) & (dfPPV.indicateur=='tmja')]
idComptagForceMax=dfPPVForce.loc[dfPPVForce.valeur==dfPPVForce.groupby('gid').valeur.transform(max)].id_comptag.unique()
dfCptAcreer=dfPPV.loc[((dfPPV.id_comptag_bdd.isna()) |( dfPPV.id_comptag.isin(idComptagForceMax)))&(dfPPV.indicateur=='tmja')]
#simple comptages supp sur compteur presents
dfCptAModifier=dfPPV.loc[((~dfPPV.id_comptag_bdd.isna()) & (~dfPPV.id_comptag.isin(idComptagForceMax))) & (dfPPV.indicateur=='tmja')]
dfCptAModifierMax=dfPPV.loc[(dfPPV.id_comptag.isin(dfCptAModifier.id_comptag.tolist())) & (dfPPV.valeur==dfPPV.groupby('gid').valeur.transform(max))]

In [366]:
compteur=dfCptAcreer[['geometrie', 'id_comptag', 'route', 'reseau', 'dep', 'gestionnai','concession', 'type_poste', 'src_geo', 'obs_geo', 'x_l93', 'y_l93',
       'fictif', 'src_cpt', 'convention','obs_supl','sens_cpt']].drop_duplicates().copy()
comptage=dfCptAcreer[['id_comptag', 'annee', 'periode']].assign(src='gestionnaire', type_veh='tv/pl').drop_duplicates()
indicAgrege=dfCptAcreer.merge(transfert.recupererIdUniqComptage(dfCptAcreer.id_comptag.tolist(), '2018'), on=['id_comptag', 'annee'])[['id_comptag_uniq', 'indicateur', 'valeur', 'fichier']]
indicHoraire=dfHoraire.loc[dfHoraire.id_comptag.isin(dfCptAcreer.id_comptag.tolist())].assign(annee='2018', 
    indicateur=dfHoraire.loc[dfHoraire.id_comptag.isin(dfCptAcreer.id_comptag.tolist())].indicateur.str.upper()).merge(transfert.recupererIdUniqComptage(
    dfHoraire.loc[dfHoraire.id_comptag.isin(dfCptAcreer.id_comptag.tolist())].id_comptag.tolist(), '2018'), on=['id_comptag', 'annee']).drop([
    'sens_cpt', 'note_manuelle_qualite', 'obs_qualite', 'annee', 'id_comptag', 'obs_supl'], axis=1)

In [372]:
len(pd.concat([dfCptAcreer,dfCptAModifierMax]))

56

In [368]:
#transférer les données
transfert.insert_bdd('local_otv_boulot', 'comptage', 'compteur', compteur)
transfert.insert_bdd('local_otv_boulot', 'comptage', 'comptage',comptage)
transfert.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',indicAgrege)
transfert.insert_bdd('local_otv_boulot', 'comptage', 'indic_horaire',indicHoraire)

In [13]:
ajout=MHCorbin(r"C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet\COMPTAGES ANGLET\Comptages 2017\données brutes\211, Promenade de la Barre.mdb")

In [9]:
dicoTables={}
with ct.ConnexionBdd('mdb', fichierMdb=r"C:\Users\martin.schoreisz\Documents\temp\OTV\Anglet\COMPTAGES ANGLET\Comptages 2017\données brutes\211, Promenade de la Barre.mdb") as c:
    tables = list(c.mdbCurs.tables())
    for k in [t[2] for t in tables if t[2].lower() not in ('msysaces','msysobjects','msysqueries','msysrelationships')] :
        dicoTables[k.lower()]=pd.read_sql(f"SELECT * FROM {k}", c.connexionMdb)

In [10]:
dicoTables['hshdr']

In [12]:
tables

[('C:\\Users\\martin.schoreisz\\Documents\\temp\\OTV\\Anglet\\COMPTAGES ANGLET\\Comptages 2017\\données brutes\\211, Promenade de la Barre.mdb', None, 'MSysACEs', 'SYSTEM TABLE', None),
 ('C:\\Users\\martin.schoreisz\\Documents\\temp\\OTV\\Anglet\\COMPTAGES ANGLET\\Comptages 2017\\données brutes\\211, Promenade de la Barre.mdb', None, 'MSysObjects', 'SYSTEM TABLE', None),
 ('C:\\Users\\martin.schoreisz\\Documents\\temp\\OTV\\Anglet\\COMPTAGES ANGLET\\Comptages 2017\\données brutes\\211, Promenade de la Barre.mdb', None, 'MSysQueries', 'SYSTEM TABLE', None),
 ('C:\\Users\\martin.schoreisz\\Documents\\temp\\OTV\\Anglet\\COMPTAGES ANGLET\\Comptages 2017\\données brutes\\211, Promenade de la Barre.mdb', None, 'MSysRelationships', 'SYSTEM TABLE', None),
 ('C:\\Users\\martin.schoreisz\\Documents\\temp\\OTV\\Anglet\\COMPTAGES ANGLET\\Comptages 2017\\données brutes\\211, Promenade de la Barre.mdb', None, 'A5', 'TABLE', None),
 ('C:\\Users\\martin.schoreisz\\Documents\\temp\\OTV\\Anglet\\COMPTA

# ***Ville Niort***
> 2020 : une dizaine de ponts de comptage. l'enjeu est sur al localisation 

In [ ]:
#creation des donnees a transferer
cpt_niort=it.Comptage_Niort(r'D:\temp\otv\2019\Donnees_source\Niort\NIORT_COMPTAGE_ROUTIER_2020\2020',2020)
dico_id_comptag={
'Niort-rue_alsace_lorraine--0,4580;46,3287' : '01 2020 Alsace Lorraine',
'Niort-boulevard_cassin--0,4523;46,3223' :'01 2020 Rue Terraudiere Bld Cassin_2',
'Niort-rue_terraudiere--0,4524;46,3228' :'01 2020 Rue Terraudiere Bld Cassin_1',
'Niort-rue_chateau_menu--0,4524;46,3592': '02 2020 Chateau Menu',
'Niort-rue_de_souche--0,4345;46,3260':'02 2020 rue de Souché',
'Niort-rue_du_24_fevrier--0,4628;46,3212':'03 2020 24 Février',
'Niort-avenue_de_paris--0,4458;46,3285':'03 2020 Paris',
'Niort-avenue_saint_jean_angely--0,4656;46,3203':'03 2020 St Jean',
'Niort-rue_14_juillet--0,4560;46,3233':'01 2020 Limoges 14 Juillet_2',
'Niort-rue_marechal_leclerc--0,4400;46,3527':'01 2020 Mal Leclerc',
'Niort-avenue_de_limoges--0,4565;46,3221':'01 2020 Limoges 14 Juillet_1',
'Niort-rue_de_la_la_gare--0,4578;46,3203':'01 2020 rue de la gare'}
dico_fichiers_final=cpt_niort.creer_dico(dico_id_comptag)
cpt_niort.horaire_tout_cpt(dico_fichiers_final)
cpt_niort.agrege_tout_cpt(dico_fichiers_final)

In [ ]:
cpt_niort.df_attr

In [ ]:
val_txt=cpt_niort.creer_valeur_txt_update(cpt_niort.df_attr, ['id_comptag','tmja','pc_pl', 'src'])
cpt_niort.update_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_p', val_txt,{f'tmja_{str(cpt_niort.annee)}':'tmja',f'pc_pl_{str(cpt_niort.annee)}':'pc_pl', f'src_{str(cpt_niort.annee)}':'src'})
cpt_niort.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_horaire', cpt_niort.df_attr_horaire)

> ## 2016 a 2019 : 
on peut creer le dico_id_comptag a partir des donnees stockes dans la table (geom, id des fchiers, reference du dossier)

In [ ]:
dossier_src=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\Niort\comptages_2016_2019\CEREMA'
annees=['2016','2017','2018','2019']
dico_dossier_src={annee:os.path.join(dossier_src, annee) for annee in annees}
#obtenir le lien entre les fichiers et de compatg et l'id_comptag, precedemmnet entre dans la bdd via qgis
with ct.ConnexionBdd('gti_otv_pg11') as c : 
    rqt="select id_comptag, id_cpt, fichier from comptage.na_2010_2019_p where id_comptag like 'Niort%%' and fichier IS NOT null"
    ids=pd.read_sql(rqt, c.sqlAlchemyConn)
#formater pour pouvoir utiliser le code creer pour 2020
dico_id_comptag={id_comptag:[os.path.join(d,f) for f in id_cpt.split(';')] for id_comptag,id_cpt,d in 
                 zip(ids.id_comptag.tolist(),ids.id_cpt.tolist(),ids.fichier.tolist()) }

In [ ]:
#calculer les données et les insérer
for a,val in dico_dossier_src.items():
    dico_limite={k:v for k,v in dico_id_comptag.items() if all([val in e for e in v])}
    cpt_niort=it.Comptage_Niort(val,a)
    cpt_niort.agrege_tout_cpt(dico_limite)
    cpt_niort.horaire_tout_cpt(dico_limite)
    cpt_niort.df_attr=cpt_niort.df_attr.loc[~cpt_niort.df_attr.tmja.isna()].copy()
    cpt_niort.df_attr_horaire=cpt_niort.df_attr_horaire.loc[cpt_niort.df_attr_horaire.id_comptag.isin(cpt_niort.df_attr.id_comptag.tolist())].copy()
    cpt_niort.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2019_horaire',cpt_niort.df_attr_horaire)

# ***Grand Dax***
> une 20aine de points de comptage. l'enjeu est sur la localisation : faite à la main pour aller plus vite

In [ ]:
cptDax=it.Comptage_GrandDax(r'D:\temp\otv\2019\donnees_produite\points_Dax_temp.shp')
cptDax.creer_df_agrege()
cptDax.df_horaire_final()

In [ ]:
cptDax.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_p', cptDax.df_attr)
cptDax.insert_bdd('local_otv_station_gti', 'comptage', 'na_2010_2019_horaire',cptDax.df_attr_horaire)

# ***Limoges Metropole***
> un Webservice est dispo ici : https://siglm.agglo-limoges.fr/servernf/rest/services/_TRANSPORTS/transports_consult/featureServer
on peut y acceder via Qgis, ce qui permet de creer un shape : Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\LIMOGES\Limoge_Web_service.shp

## isoler les points représentatifs du trafic total
1. ajouter des attributs d'identification : de la représentativité, de groupement
1. Checker la validité des attributs et déterminer sur lesquels s'appuyer
1. Regrouper les points avec un identfiant
1. Conserver les points avec 'type' == 'Double' ie 'representatif'=True

In [ ]:
#POUR INFO
#rechreche incohérence entre direction et type : direction 2 sens et type Simple
limMet.loc[(limMet.apply(lambda x : any([a.lower() in x['direction'].lower() 
            for a in ('2 sens','2  sens', 'deux sens', 'double sens', 'cumul') if x['direction']]), axis=1)) & (limMet['type']=='Simple')].objectid.tolist()
"""objectId in (40390.0, 40844.0, 41639.0, 42174.0, 42175.0, 42176.0, 42177.0, 42432.0, 42599.0, 43066.0, 43720.0, 
52077.0, 39017.0, 39274.0, 39586.0, 39959.0, 40014.0, 40015.0) prioriser 'type'=='Double', ne pas trop s'appuyer sur le type"""
#rechreche incohérence entre direction et type : direction != 2 sens et type Double : là aussi, ça confirme de s'appuyer sur le type Double
limMet.loc[(limMet.apply(lambda x : all([a.lower() not in x['direction'].lower() 
            for a in ('2 sens','2  sens', 'deux sens', 'double sens', 'cumul') if x['direction']]), axis=1)) & (limMet['type']=='Double')].objectid.tolist()

In [ ]:
limMet=it.Comptage_Limoges_Metropole(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\LIMOGES\Limoge_Web_service.shp',
                                    r'D:\temp\otv\2019\donnees_produite\test_limoges\zone_equi_cpt.shp')
limMet.groupe_point()
limMet.df_regroupee_complete(limMet.df_regroupee())
limMet.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2019_p', limMet.df_attr)

# ***Agglo LaRochelle***
> bcp de donnée éparpillées u peu n'importe comment ici : <br> Chaque commune à sa façon de bosser...

> ## Exemple sainte-SOulle

In [ ]:
dossier=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\La_Rochelle\donnees-la-rochelle\Données La Rochelle\7-Communes\SteSoulle'
toto=it.Comptage(os.path.join(dossier,'rue des Fortiness.txt'))
SteSoulle=pd.read_csv(os.path.join(dossier,'rue des Fortines.txt'), sep=' ')
SteSoulle.rename(columns={'Nombrevéhicules':'nbveh', 'Date':'heure'}, inplace=True)
SteSoulle.reset_index(inplace=True)
SteSoulle.rename(columns={'Nombrevéhicules':'nbveh', 'Date':'heure', 'index':'jour'}, inplace=True)
SteSoulle.set_index(SteSoulle.apply(lambda x : pd.to_datetime(x['jour']+ ' ' + x.heure, dayfirst=True), axis=1), inplace=True)
#les données sur le 4,10 et 11 février sont bizarres, on les exclus,du coup pour ne pas fausser le tmja avec trop de WE je ne garde que les jours ouvrés et je fais un tmjo
#SteSoulle=SteSoulle.loc[(~SteSoulle.index.dayofyear.isin([pd.to_datetime(x).dayofyear for x in (('2016-03-28','2016-03-23','2016-04-02'))]))
                       #& (~SteSoulle.index.dayofweek.isin((5,6)))][['nbveh', 'jour', 'heure']].copy()

In [ ]:
#calcul tmja (basé sur calcul_indicateurs_agreges() de la classe FIM)
df_jour=SteSoulle[['nbveh']].resample('1D').sum()
df_jour=df_jour.loc[df_jour['nbveh']!=0].copy()
if len(df_jour)<7 : 
    raise it.PasAssezMesureError(len(df_jour))
elif len(df_jour) in (7,8) : 
    traf_list=df_jour.nbveh.tolist()
    tmjo=int(statistics.mean([traf_list[0]+traf_list[-1]]+traf_list[1:-1]))
else : 
    tmjo=int(df_jour.iloc[1:-1].nbveh.mean())*2

In [ ]:
df_horaire=SteSoulle[['jour', 'heure', 'nbveh']].pivot(index='jour', columns='heure', values='nbveh').fillna(0)*2
df_horaire.columns=[f'h{str(int(c.split(":")[0]))}_{str(int(c.split(":")[0])+1)}' for c in df_horaire.columns]
df_horaire=df_horaire.reset_index().assign(type_veh='TV', id_comptag='LaRoche-r des fortines--1.0391;46.188')

In [ ]:
toto.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2019_horaire', df_horaire)

> ## La Rochelle
plusieurs dossier contenant desfichiers de comptages, : Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\La_Rochelle\campagne-comptages-2015\trafic_aggloLR_SMOB2015 ; Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\La_Rochelle\comptages_delattre

In [ ]:
Larochelle.update_bdd_LaRochelle('gti_otv_pg11', 'comptage', 'na_2010_2019_p')
Larochelle.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2019_horaire', Larochelle.df_attr_horaire)

In [ ]:
#prepar donnees
Larochelle=it.Comptage_LaRochelle(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\La_Rochelle\donnees-la-rochelle\Données La Rochelle\7-Communes\LaRochelle_LaPalice')
ids=Larochelle.listCptCreer()
Larochelle.creer_dfs(ids)

In [ ]:
with ct.ConnexionBdd('gti_otv_pg11') as c : 
            rqt="select id_comptag, id_cpt from comptage.na_2010_2019_p where id_comptag in ('LaRoch-r marcel deflandre--1.2128;46.1698','LaRoch-r de bethencourt--1.2028;46.1712','LaRoch-r de bethencourt--1.2014;46.172')"
            ids=pd.read_sql(rqt, c.sqlAlchemyConn)

In [ ]:
fichier=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\La_Rochelle\donnees-la-rochelle\Données La Rochelle\7-Communes\LaRochelle_LaPalice\SEMAINE 2\P1-RUE MARCEL DEFLANDRE SENS 2 SEMAINE 2.xls'

In [ ]:
df=pd.read_excel(fichier)
tmja=round(df.loc[38,'Unnamed: 6'])
pl=round(df.loc[42,'Unnamed: 6'])
periode=df.iloc[1,16]
index_date=pd.date_range(start=pd.to_datetime(periode.split(' au ')[0][3:], dayfirst=True), end=pd.to_datetime(periode.split(' au ')[1], dayfirst=True))
df_pl=df.iloc[17:24,2:26].copy()
df_tv=df.iloc[27:34,2:26].copy()
df_pl.columns=[f'h{str(i)}_{str(i+1)}' for i in range(24)]
df_tv.columns=[f'h{str(i)}_{str(i+1)}' for i in range(24)]
df_pl=df_pl.assign(jour=index_date, type_veh='PL')
df_tv=df_tv.assign(jour=index_date, type_veh='TV')

<a id="7" ></a>
# ***DIRA***

## Année 2019
> il existe trois sorte de données, TMJA, TMJM et horaires. les données TMJA DOIVENT etre issu de (ordre de priorité) :<br>
1. de la carto DIRA si le point est représenté
1. du fichier global (Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRA\0_tmja_dira_par_section_20200106.ods)
1. des fichiers horaires 
<br> les données horaires ne peuvent etre issue que des fichiers horaires Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRA\0_Annee_Complete_2019
- des graphs dispo dans le notebook Graph_tafics
- attention, certaines données de TMJA sont mises à jour apres prises en comptes des données horares,,quand une données horaire est dispo et le TMJA non. fait sous sql  
***ATTENTION : EN 2020 IL Y  A EU CONFUSION NETRE LES POINTS rn150 MEDIS ET ST-ROMAIN-DE-BENET / BIEN CHECKER QUE TOUS LES IDENTFIANTS LIES AU BOUCLE ET AUTRES SONT OK***

> ## *TMJA / TMJM*

In [ ]:
#creation de l'objet
dira=it.Comptage_Dira(r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRA\0_tmja_dira_par_section_20200106.ods',
                      r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRA\0_Annee_Complete_2019',
                     '2019','gti_otv_pg11', 'na_2010_2019_p')

In [ ]:
#mise en form edes données
dfMensGrp=dira.MiseEnFormeMensuelleAnnuelle(dira.verifValiditeMensuelle(dira.jointureExistantFichierTmja('gti_otv_pg11', 'na_2010_2019_p')))
dira.MiseEnFormeAnnuelle(dfMensGrp,'16-N10-2+700')
dira.MiseEnFormeMensuelle(dfMensGrp,'16-N10-2+700')
#mise à jour de la Bdd
dira.update_bdd_Dira('gti_otv_pg11', 'comptage', 'na_2010_2019_p', nullOnly=True)
dira.insert_bdd('gti_otv_pg11','comptage', 'na_2010_2019_mensuel', dira.df_attr_mens)

In [3]:
#2020
dira=it.Comptage_Dira(r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\0_tmja_dira_par_section_20210101.ods',
                      r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\GrosFichiers - JP CASSOU\0_Annee_Complete_2020',
                      r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\dira_tmja_2020.ods',
                     '2020','local_otv_boulot', 'compteur')

In [110]:
#mise en forme des données 
dfMensGrp=dira.MiseEnFormeMensuelleAnnuelle(dira.verifValiditeMensuelle(dira.jointureExistantFichierTmja()))
dira.MiseEnFormeAnnuelle(dfMensGrp)
dira.MiseEnFormeMensuelle(dfMensGrp)
#insertion des comptages
dira.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dira.creer_comptage(dira.df_attr.id_comptag.unique(), dira.annee, 'tableau annuel DIRA', 'tv/pl'))
#insertion des indicateurs agreges
dira.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',dira.structureBddOld2NewForm(dira.df_attr, dira.annee,['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))
#insertion des indicateurs mensuel
dira.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',dira.structureBddOld2NewForm(dira.df_attr_mens.replace(-99, np.nan), 
            dira.annee,['id_comptag', 'annee', 'fichier', 'donnees_type'],['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept',
       'octo', 'nove', 'dece'], 'mensuel'))

In [370]:
#gestion des données carto
donneesAgregeesUpdate, donneesAgregeesInsert=dira.cptCartoInsertUpdate()

In [207]:
#update comptages
dira.update_bdd('comptage', 'comptage', dira.creer_valeur_txt_update(donneesAgregeesUpdate.loc[donneesAgregeesUpdate.obs.isna()], ['id', 'src']),
                {'src':'src'}, identifiant='id')
dira.update_bdd('comptage', 'comptage', dira.creer_valeur_txt_update(donneesAgregeesUpdate.loc[~donneesAgregeesUpdate.obs.isna()], ['id', 'src', 'obs']),
                {'src':'src', 'obs':'obs'}, identifiant='id')
#update tmja
dira.update_bdd('comptage', 'indic_agrege', dira.creer_valeur_txt_update(donneesAgregeesUpdate.assign(id_comptag_uniq=donneesAgregeesUpdate.id)
                                                                         , ['id_comptag_uniq', 'tmja']), {'valeur':'tmja'}, identifiant='id_comptag_uniq',
                filtre = "indicateur='tmja'")
#update pc_pl
dira.update_bdd('comptage', 'indic_agrege', dira.creer_valeur_txt_update(donneesAgregeesUpdate.assign(id_comptag_uniq=donneesAgregeesUpdate.id)
                                                                         , ['id_comptag_uniq', 'pc_pl']), {'valeur':'pc_pl'}, identifiant='id_comptag_uniq',
                filtre = "indicateur='pc_pl'")

In [225]:
#inserer comptages
dira.insert_bdd('local_otv_boulot', 'comptage', 'comptage',dira.creer_comptage(donneesAgregeesInsert.id_comptag.unique(), dira.annee, 'carto dira 2020', 'tv/pl', obs=donneesAgregeesInsert.obs.tolist()))
dira.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',dira.structureBddOld2NewForm(donneesAgregeesInsert, dira.annee,['id_comptag', 'annee', 'fichier'],['tmja', 'pc_pl'], 'agrege'))

> ## *Donnees horaires*

In [ ]:
%time dfTousFichier,idCptNonAffectes,dblATraiter=dira.concatTousFichierHoraire()

In [ ]:
#vérifier
dblATraiter
idCptNonAffectes

In [6]:
#Séparer les données horaires entres comptages existants et comptages a creer
dfHoraireIdConnus, dfHoraireIdsInconnus=dira.scinderComptagExistant(dfTousFichier, dira.annee, table='comptage')
dfHoraireIdsInconnus.rename(columns={'type_veh':'indicateur'}, inplace=True)

In [24]:
#Comptages a creer
#mise en forme des donnees horaire vers du tmja
dfTmjaPcpl=tmjaDepuisHoraire(dfHoraireIdsInconnus)
dfMeltInconnusPeriode=periodeDepuisHoraire(dfHoraireIdsInconnus)

#scinder les tablesvers comptages et indics_agrege
#creer les comptages manquants puis recuperer les ids correspondants
dfToutTable=dfTmjaPcpl.merge(dfMeltInconnusPeriode[['id_comptag', 'periode']], on=['id_comptag']).assign(
    src='tmja reconstitue depuis donnees horaires 2020', type_veh='tv/pl')
dfComptages=dfToutTable[['id_comptag','annee', 'periode', 'src', 'type_veh']].drop_duplicates(['id_comptag','annee', 'periode', 'src', 'type_veh'])
dira.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dfComptages)
dfIdCptUniqsNew=dira.recupererIdUniqComptage(dfComptages.id_comptag.tolist(), '2020', 'local_otv_boulot')

In [43]:
#inserer les indicateurs agreges
dfIndicAgreges=dfToutTable[['id_comptag', 'annee', 'indicateur', 'valeur']].merge(dfHoraireIdsInconnus.assign(fichier=dfHoraireIdsInconnus.fichier.apply(
    lambda x : ';'.join(set(x.split('.xls'))).strip(';'))).groupby('id_comptag').fichier.agg(lambda x : ';'.join(set(tuple(x)))).reset_index(), on='id_comptag').merge(
    dfIdCptUniqsNew, on=['id_comptag','annee']).rename(columns={'id':'id_comptag_uniq'}).drop(['id_comptag', 'annee'], axis=1)
dira.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege', dfIndicAgreges)

In [55]:
#refaire tourner  l'identification des comptages existant ou non (nromalement tout le monde defvrait etre existant) puis inserer
dfHoraireIdConnus, dfHoraireIdsInconnus=dira.scinderComptagExistant(dfTousFichier, dira.annee, table='comptage')
dira.insert_bdd('local_otv_boulot', 'comptage', 'indic_horaire',dfHoraireIdConnus.drop(['id_comptag', 'index', 'annee'], axis=1).assign(fichier=dfHoraireIdConnus.fichier.apply(
    lambda x : ';'.join(set(x.split('.xls'))).strip(';'))).rename(columns={'type_veh':'indicateur'}))

## Année 2020

**NOUVELLE SOURCE DE DONNEES VIA Christophe Damas et DtecTV**
on va aussi chercher à obtenir les données DIRA, mais d'abord on puet checker celles contenues dans les fichiersde TV ici : 
Q:\DAIT\TI\DREAL33\2021\OTV\Doc_travail\Donnees_sources\DIRA\Donnees_Tipi_TV

In [4]:
donneesMinutes=pd.read_csv(r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', delimiter=',')

In [39]:
#liste des stations TIPI
listIdCompteurTipi=donneesMinutes.PME_ID.unique()
#liste des stations OTV
with ct.ConnexionBdd('local_otv_boulot') as c : 
    listObsSupl=pd.read_sql("select distinct id_comptag, id_cpt, obs_supl from comptage.compteur where gestionnai='DIRA'", c.sqlAlchemyConn)
listStationOtvBrut=listObsSupl.obs_supl.apply(lambda x : x.split(';')[1].split('station : ')[1] if not pd.isnull(x) and 'EMC' not in x and ';' in x else x).unique()
listStationOtvBrut2=[e.split(',') for e in listStationOtvBrut if e and re.search('^M.*\.._.*$', e)]
listStationOtv=[a.strip().replace('_','') for b in listStationOtvBrut2 for a in b]

In [41]:
#station TIPI non présentes dans OTV : 
StationTipiHorsOTV=[s for s in listIdCompteurTipi if s not in listStationOtv]
StationOtvHorsTipi=[s for s in listStationOtv if s not in listIdCompteurTipi]
StationOtvTipi=[s for s in listIdCompteurTipi if s in listStationOtv]

<a id="8" ></a>
# ***DIRCO***
- Année 2019 : 
> il existe trois sorte de données, TMJA, TMJM et horaires. les données TMJA et TMJM chaque source de données à sonpropre fichier

In [2]:
#2019
fichierMja=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRCO\TMJA DIRCO-NA 2019_unfused.csv'
fichierMjM=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRCO\TMJM DIRCO-NA 2019.ods'
dossierHoraire=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\DIRCO\données dirco'
dirco=it.Comptage_Dirco(fichierMja,fichierMjM,dossierHoraire,'2019')

In [3]:
#2020
fichierMja=r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRCO\TMJA DIRCO 2020 NA.ods'
fichierMjM=r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRCO\TMJM DIRCO 2020 NA.ods'
dossierHoraire=r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRCO\Re_ Observatoire des trafics routiers DREAL NA 2020'
dirco=it.Comptage_Dirco(fichierMja,fichierMjM,dossierHoraire,'2020')

> ## TMJA

In [115]:
#import des données et mise en forme des données
dfTrafic=dirco.miseEnFormeMJA()

In [106]:
#verifier et creer des compteurs si besoin
dfIdsConnus, dfIdsInconnus=dirco.scinderComptagExistant(dfTrafic, '2020', gest='DIRCO')
#modfi de la mise en forme
dfIdsConnus['obs_supl']=dfIdsConnus.merge(dirco.existant[['obs_supl', 'id_comptag']], on='id_comptag').apply(lambda x : x['obs_supl_y']+';'+x['obs_supl_x'] if x['obs_supl_y'] else x['obs_supl_x'], axis=1 )

In [92]:
#MaJ de l'obs_supl
dirco.update_bdd('comptage', 'compteur', dirco.creer_valeur_txt_update(dfIdsConnus.loc[~dfIdsConnus.obs_supl.isna()], ['id_comptag', 'obs_supl']), {'obs_supl':'obs_supl'})
#creation des comptages
dirco.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dirco.creer_comptage(dfIdsConnus.id_comptag.tolist(), dirco.annee, 'tableau TMJA DIRCO', 'tv/pl'))

In [108]:
#insertion des données agrégées
dirco.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege', 
                 dirco.structureBddOld2NewForm(dfIdsConnus, '2020', ['id_comptag', 'fichier', 'annee'], ['tmja', 'pc_pl'], 'agrege'))

> ## TMJM

In [97]:
#2019
dfMensuel=dirco.indicateurGlobalFichierMJM('gti_otv_pg11','3-N145-9+573')

In [99]:
dirco.insert_bdd('gti_otv_pg11','comptage', 'na_2010_2019_mensuel', dfMensuel)

In [367]:
#2020
dfMensuel=dirco.indicateurGlobalFichierMJM('local_otv_boulot', [])

In [368]:
#trouver les comptages inconnus en 2020, verifier qu'il ne manque pas de compteur (termine a la main)
dfIdsConnus, dfIdsInconnus=dirco.scinderComptagExistant(dfMensuel, '2020', 'comptage', gest='DIRCO')
dfIdsInconnus.loc[~dfIdsInconnus.id_comptag.isin(dirco.existant.id_comptag.tolist())]

,id_comptag,janv,fevr,mars,avri,mai,juin,juil,aout,sept,octo,nove,dece,donnees_type,obs,annee,id_comptag_uniq


In [232]:
#creer les commptages inconnus (que si les compteurs existent)
dirco.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dirco.creer_comptage(dfIdsInconnus.id_comptag.unique(), dirco.annee, 'tableau TMJM DIRCO', 'tv/pl', obs='1 seul sens disponible'))

In [359]:
#trouver les comptages ayant des donnees mensuelles mais pas de donnees TMJA
dfCptSansTmja=dirco.recupererComptageSansTrafic(dfIdsConnus.id_comptag.tolist(), '2020')
dfCptSansTmja2=dfIdsConnus.loc[dfIdsConnus.id_comptag.isin(dfCptSansTmja.id_comptag.tolist())].copy()

In [ ]:
#calculer les indicateurs agreges des comptages sans tmja
dfCptSansTmja2['tmja']=dfCptSansTmja2.loc[dfCptSansTmja2.donnees_type=='tmja'].apply(lambda x : int(mean([x[e] for e in dfCptSansTmja2.columns if e not in ('id_comptag', 'donnees_type', 'annee', 'id_comptag_uniq', 'obs')])), axis=1)
dfCptSansTmja2['pc_pl']=dfCptSansTmja2.loc[dfCptSansTmja2.donnees_type=='pc_pl'].apply(lambda x : round(mean([x[e] for e in dfCptSansTmja2.columns if e not in ('id_comptag', 'donnees_type', 'annee', 'id_comptag_uniq', 'obs') and not pd.isnull(x[e])]),2), axis=1)
#dfCptSansTmja2['valeur']=dfCptSansTmja2.apply(lambda x : x['tmja'] if not pd.isnull(x['tmja']) else x['pc_pl'], axis=1)

In [373]:
#inserer
#agerege
dirco.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege', dfCptSansTmja2.rename(columns={'donnees_type':'indicateur'}).assign(fichier=os.path.basename(dirco.fichierTmjm))[['id_comptag_uniq','indicateur','valeur','obs', 'fichier']])
#mensuel
dirco.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel', dirco.structureBddOld2NewForm(dfIdsConnus.assign(fichier=os.path.basename(dirco.fichierTmjm)), 
                        dirco.annee, ['id_comptag', 'annee','obs', 'fichier', 'donnees_type'], ['janv', 'fevr', 'mars', 'avri', 'mai', 'juin', 'juil', 'aout', 'sept',
       'octo', 'nove', 'dece'], 'mensuel' ))

> ## Horaire
on ne dispose que de donnees TV, et il faut au prealable joindre les id_comptagavec les code_sation des donnes horaire. POur ça on utilise le fichier de tmja comme pivot

In [8]:
dfFichierTmja=dirco.miseEnFormeFichierTmjaPourHoraire()

In [ ]:
dicoCptOkTot,dicoCptErrorTot=dirco.tousFichierHoraires(dfFichierTmja)
dicoCptErrorTot

In [38]:
#trouver les comptages inconnus en 2020, verifier qu'il ne manque pas de compteur (termine a la main)
dirco.corresp_nom_id_comptag(dirco.df_attr_horaire)
dfIdsConnus, dfIdsInconnus=dirco.scinderComptagExistant(dirco.df_attr_horaire, '2020', 'comptage', gest='DIRCO')
dfIdsInconnus.loc[~dfIdsInconnus.id_comptag.isin(dirco.existant.id_comptag.tolist())]

,jour,id_comptag,fichier,h0_1,h1_2,h2_3,h3_4,h4_5,h5_6,h6_7,h7_8,h8_9,h9_10,h10_11,h11_12,h12_13,h13_14,h14_15,h15_16,h16_17,h17_18,h18_19,h19_20,h20_21,h21_22,h22_23,h23_24,type_veh,annee,id_comptag_uniq


In [27]:
dfIdsInconnus

,jour,id_comptag,fichier,h0_1,h1_2,h2_3,h3_4,h4_5,h5_6,h6_7,h7_8,h8_9,h9_10,h10_11,h11_12,h12_13,h13_14,h14_15,h15_16,h16_17,h17_18,h18_19,h19_20,h20_21,h21_22,h22_23,h23_24,type_veh,annee,id_comptag_uniq
30,2021-09-01,19-A20-276+0,a20 19.xlsx,275.0,254.0,222.0,218.0,242.0,353.0,661.0,1294.0,1559.0,1518.0,1953.0,2046.0,1733.0,1498.0,1530.0,1692.0,1366.0,1731.0,1462.0,1081.0,731.0,632.0,442.0,262.0,TV,2020,NaN
31,2021-09-02,19-A20-276+0,a20 19.xlsx,220.0,225.0,224.0,213.0,233.0,359.0,611.0,1216.0,1462.0,1702.0,1934.0,2102.0,1731.0,1588.0,1903.0,1744.0,1866.0,1891.0,1571.0,1156.0,770.0,597.0,430.0,280.0,TV,2020,NaN
32,2021-09-03,19-A20-276+0,a20 19.xlsx,226.0,236.0,218.0,213.0,253.0,339.0,596.0,1149.0,1550.0,1658.0,2102.0,2215.0,1965.0,1877.0,1964.0,1805.0,1672.0,1813.0,1825.0,1749.0,1225.0,848.0,541.0,385.0,TV,2020,NaN
33,2021-09-04,19-A20-276+0,a20 19.xlsx,283.0,219.0,224.0,238.0,243.0,290.0,453.0,823.0,1347.0,2029.0,2465.0,2358.0,2198.0,1923.0,1977.0,1772.0,1536.0,1325.0,1011.0,957.0,578.0,409.0,309.0,265.0,TV,2020,NaN
34,2021-09-05,19-A20-276+0,a20 19.xlsx,198.0,162.0,109.0,93.0,97.0,139.0,231.0,426.0,728.0,1357.0,1880.0,2135.0,1731.0,1536.0,1873.0,1903.0,1870.0,1812.0,1544.0,1294.0,933.0,638.0,392.0,275.0,TV,2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,2021-09-26,87-N21-6+947,rn21 221 1021 1113.xlsx,129.0,76.0,66.0,25.0,22.0,21.0,74.0,115.0,156.0,281.0,347.0,444.0,437.0,351.0,449.0,457.0,561.0,605.0,579.0,370.0,255.0,144.0,74.0,41.0,TV,2020,NaN
850,2021-09-27,87-N21-6+947,rn21 221 1021 1113.xlsx,11.0,8.0,13.0,27.0,66.0,145.0,325.0,989.0,1038.0,583.0,540.0,530.0,594.0,644.0,637.0,578.0,850.0,1001.0,835.0,497.0,335.0,194.0,99.0,41.0,TV,2020,NaN
851,2021-09-28,87-N21-6+947,rn21 221 1021 1113.xlsx,20.0,11.0,17.0,36.0,88.0,146.0,294.0,987.0,1050.0,595.0,514.0,528.0,604.0,732.0,622.0,660.0,813.0,1060.0,878.0,605.0,361.0,188.0,118.0,72.0,TV,2020,NaN
852,2021-09-29,87-N21-6+947,rn21 221 1021 1113.xlsx,28.0,15.0,17.0,34.0,84.0,141.0,319.0,945.0,988.0,611.0,585.0,548.0,740.0,707.0,699.0,652.0,825.0,976.0,855.0,593.0,324.0,190.0,100.0,53.0,TV,2020,NaN


In [28]:
#creer les commptages inconnus (que si les compteurs existent)
dirco.insert_bdd('local_otv_boulot', 'comptage', 'comptage', dirco.creer_comptage(dfIdsInconnus.id_comptag.unique(), dirco.annee, 'donnees horaires DIRCO', 'tv', obs='tmja recalcule a partirdes donnees horaires'))

In [39]:
#trouver les comptages ayant des donnees horaires mais pas de donnees TMJA
dfCptSansTmja=dirco.recupererComptageSansTrafic(dfIdsConnus.id_comptag.tolist(), '2020')
dfCptSansTmja2=dfIdsConnus.loc[dfIdsConnus.id_comptag.isin(dfCptSansTmja.id_comptag.tolist())].copy()

In [58]:
#mettre en forme les données
dfTmja=dfCptSansTmja2[['id_comptag_uniq', 'jour','fichier']+attributsHoraire].assign(tmj=lambda x : sum([x[e] for e in dfCptSansTmja2.columns if e in attributsHoraire])).groupby(['id_comptag_uniq', 'fichier']
    ).agg(trafSum=pd.NamedAgg(column='tmj',aggfunc='sum'), 
        nbJour=pd.NamedAgg(column='jour',aggfunc='count'))
dfTmja['valeur']=round(dfTmja['trafSum']/dfTmja['nbJour'])
dfTmja['indicateur']='tmja'
dfTmja['obs']='tmja recalcule depuis donnees horaires'

In [62]:
#inserer les donnes agregees non connue
dirco.insert_bdd('local_otv_boulot', 'comptage', 'indic_agrege',dfTmja.drop(['trafSum', 'nbJour'], axis=1).reset_index())

In [69]:
#inserer les donnes horaires
dirco.insert_bdd('local_otv_boulot', 'comptage', 'indic_horaire',dirco.structureBddOld2NewForm(dirco.df_attr_horaire, dirco.annee, ['id_comptag', 'annee'], ['toto'], 'horaire'))

<a id="9" ></a>
# ***DIRSO***
- Année 2020 : 
> il n'y a que 2 points, dont les données sont téléchargées directement sur le site de la DIRCO (cf src.txt sur Box)
les 2 seuls poijnts qui nous interessent sont MAZERES N524 8+890 et  LAPEYRADE N524 64+800

In [52]:
ficherMensuel=r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\DIRSO\en_cours\hors_convention\tmja_2020_cumule.xls'
dirso=it.Comptage(ficherMensuel)
dfMens=pd.read_excel(dirso.fichier, skiprows=9)

In [53]:
dfMensSo=dfMens.iloc[sorted([a for a in dfMens.loc[dfMens['Intitulé du PM'].isin(('MAZERES', 'LAPEYRADE'))].index]+[a-1 for a in dfMens.loc[dfMens['Intitulé du PM'].isin(
    ('MAZERES', 'LAPEYRADE'))].index])].copy()
dfMensSo

,Intitulé du PM,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,C. A.,Route,Janvier,Unnamed: 8,Février,Unnamed: 10,Mars,Unnamed: 12,Avril,Unnamed: 14,Mai,Unnamed: 16,Juin,Unnamed: 18,Juillet,Unnamed: 20,Août,Unnamed: 22,Septembre,Unnamed: 24,Octobre,Unnamed: 26,Novembre,Unnamed: 28,Décembre,Unnamed: 30,MJA,Unnamed: 32,Evts
93,33,932.0,0.0,C,NaN,NaN,N524,8682,NaN,9067,NaN,5418,NaN,2830,NaN,5839,NaN,8684,NaN,8997,NaN,8813,NaN,9202,NaN,8842,NaN,6196,NaN,8009,NaN,7549,NaN,NON
94,MAZERES,NaN,NaN,NaN,NaN,,8+ 890,0.055,lg,0.057,lg,0.065,lg,0.089,lg,0.069,lg,0.07,lg,0.072,lg,0.068,lg,0.072,lg,0.063,lg,0.076,lg,0.057,lg,0.066,lg,NaN
95,40,524.0,2.0,C,NaN,NaN,N524,902,NaN,922,NaN,614,NaN,329,NaN,773,NaN,1035,NaN,1174,NaN,1148,NaN,1057,NaN,1008,NaN,578,NaN,762,NaN,860,NaN,NON
96,LAPEYRADE,NaN,NaN,NaN,NaN,,64+ 800,0.159,lg,0.146,lg,0.174,lg,0.31,lg,0.168,lg,0.17,lg,0.134,lg,0.163,lg,0.379,lg,0.361,lg,0.419,lg,0.365,lg,0.235,lg,NaN


In [54]:
dfMensSoTmja=dfMensSo.loc[~dfMensSo.Evts.isna()][[a for b in it.dico_mois.values() for a in b if a in dfMensSo.columns]].assign(
    donnees_type='tmja', id_comptag=['33-N524-8+890','40-N524-64+800'],fichier=os.path.basename(ficherMensuel), annee='2020').T.drop_duplicates().T
dfMensSoPcpl=dfMensSo.loc[dfMensSo.Evts.isna()][[a for b in it.dico_mois.values() for a in b if a in dfMensSo.columns]].applymap(lambda x : x*100).assign(
    donnees_type='pc_pl', id_comptag=['33-N524-8+890','40-N524-64+800'],fichier=os.path.basename(ficherMensuel), annee='2020').T.drop_duplicates().T

In [60]:
dirso.insert_bdd('local_otv_boulot', 'comptage', 'indic_mensuel',
                 dirso.structureBddOld2NewForm(pd.concat([dirso.renommerMois(dfMensSoPcpl),dirso.renommerMois(dfMensSoTmja)]), '2020', ['id_comptag', 'annee', 'donnees_type'], 
                                               [k for k in it.dico_mois.keys()], 'mensuel'))

In [74]:
pd.read_json(dfMensSo.loc[~dfMensSo.Evts.isna()][[a for b in it.dico_mois.values() for a in b if a in dfMensSo.columns]].T.drop_duplicates().T.to_json())

,Janvier,Février,Mars,Avril,Mai,Juin,Juillet,Août,Septembre,Octobre,Novembre,Décembre
93,8682,9067,5418,2830,5839,8684,8997,8813,9202,8842,6196,8009
95,902,922,614,329,773,1035,1174,1148,1057,1008,578,762


# ***TESTS***

In [ ]:
## DETAILS OPERATIONS CD87

#regrouper les différentes données issues de ficiers et les ajouter au dico
for k, v in d87.dico_voie.items() : 
    for i,e in enumerate(v) : 
        if len(e['fichiers'])==1 : 
            print(e['fichiers'][0])
            obj_fim=it.FIM(os.path.join(d87.dossier,e['fichiers'][0]))
            try : 
                obj_fim.resume_indicateurs()
            except obj_fim.fim_PasAssezMesureError : 
                continue
            except Exception as ex : 
                print(f"erreur : {ex} \n dans fichier : {e['fichiers'][0]}")
            e['tmja'], e['pc_pl'], e['date_debut'], e['date_fin']=obj_fim.tmja, obj_fim.pc_pl, obj_fim.date_debut,obj_fim.date_fin
        elif len(e['fichiers'])>1 :
            list_tmja=[]
            list_pc_pl=[]
            for f in e['fichiers'] : 
                obj_fim=it.FIM(os.path.join(d87.dossier,f))
                print(f)
                try : 
                    obj_fim.resume_indicateurs()
                except (obj_fim.fim_PasAssezMesureError,obj_fim.fimNbBlocDonneesError)  : 
                    continue
                except Exception as ex : 
                    print(f"erreur : {ex} \n dans fichier : {f}")
                list_tmja.append(obj_fim.tmja)
                list_pc_pl.append(obj_fim.pc_pl)
            e['tmja'], e['pc_pl'], e['date_debut'], e['date_fin']=int(mean(list_tmja)), round(mean(list_pc_pl),2),np.NaN, np.NaN

#renseigner le type de poste
for k, v in d87.dico_voie.items() : 
    for e in v : 
        if len(e['fichiers']) > 4 :
            e['type_poste']='permanent'
        elif 1<len(e['fichiers'])<=4 : 
            e['type_poste']='tournant'
        elif len(e['fichiers'])== 1 :
            e['type_poste']='ponctuel'
        else : 
            e['type_poste']='NC'

#faire une df avec les points de comptage
d87.df_attr=pd.DataFrame([[k, e['pr'], e['abs'], e['tmja'], e['pc_pl'], e['type_poste'],
                      e['date_debut'],e['date_fin']] for k, v in d87.dico_voie.items() for e in v if 'tmja' in e.keys()], 
             columns=['route','pr','absc','tmja','pc_pl','type_poste','date_debut','date_fin'])
d87.df_attr['id_comptag']=d87.df_attr.apply(lambda x :'87-'+x['route']+'-'+str(x['pr'])+'+'+str(x['absc']), axis=1)

#filtre cpt ponctuel
d87.df_attr=d87.df_attr.loc[d87.df_attr.apply(lambda x : x['date_debut'].month not in [7,8] and x['date_fin'].month not in [7,8], 
                                                         axis=1)].copy()

#fare le tri avec les comptages existants : 
#recuperer les compmtages existants
d87.comptag_existant_bdd('gti_otv_pg11', 'na_2010_2018_p', schema='comptage',dep='87', type_poste=False)
d87.df_attr_update=d87.df_attr.loc[d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
d87.df_attr_insert=d87.df_attr.loc[~d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
#obtenir une cle de correspondace pour les comptages tournants et permanents
df_correspondance=d87.corresp_old_new_comptag('gti_otv_pg11', 'public','d87_cpt_temp','lineaire.traf2017_bdt87_ed17_l',
                                'referentiel', 'troncon_route_bdt87_ed17_l','troncon_route_bdt87_ed17_l_vertices_pgr','id')

#verifier si cette clé n'existent pas deja dans la table de correspondance et passer les nouvelles dedans
rqt_corresp_comptg='select * from comptage.corresp_id_comptag'
with ct.ConnexionBdd('gti_otv_pg11') as c:
    corresp_comptg=pd.read_sql(rqt_corresp_comptg, c.sqlAlchemyConn)
df_correspondance=df_correspondance.loc[~df_correspondance['id_comptag'].isin(corresp_comptg.id_gest.tolist())]
if not df_correspondance.empty():
    d87.insert_bdd('gti_otv_pg11', 'comptage', 'corresp_id_comptag', 
               df_correspondance.rename(columns={'id_comptag_lin':'id_gti','id_comptag':'id_gest'})[['id_gest','id_gti']])

#faire la correspondance entre les noms de comptage
d87.corresp_nom_id_comptag('gti_otv_pg11',d87.df_attr)

#recalculer les insert et update
d87.df_attr_update=d87.df_attr.loc[d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
d87.df_attr_insert=d87.df_attr.loc[~d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()

#mettre en forme pour update
d87.df_attr_update['obs']=d87.df_attr_update.apply(lambda x : x['date_debut'].strftime('%d/%m/%Y')+'-'+ x['date_fin'].strftime('%d/%m/%Y') if not pd.isnull(x['date_debut']) else '', axis=1)
d87.df_attr_update.loc[d87.df_attr_update.pc_pl.isna(),'obs']='pc_pl inconnu'
d87.df_attr_update.loc[d87.df_attr_update.pc_pl.isna(),'pc_pl']=-99

#preparer update
valeurs_txt=d87.creer_valeur_txt_update(d87.df_attr_update, ['id_comptag','tmja','pc_pl','obs'])
dico_attr={'tmja_2018':'tmja','pc_pl_2018':'pc_pl','obs_2018':'obs'}
#update
d87.update_bdd('gti_otv_pg11', 'comptage', 'na_2010_2018_p', valeurs_txt,dico_attr)

#mettre en forme le insert
dbl=d87.df_attr_insert.loc[d87.df_attr_insert.duplicated('id_comptag', False)].copy()
ss_dbl=d87.df_attr_insert.loc[~d87.df_attr_insert.index.isin(dbl.index.tolist())].copy()
dbl=dbl.dropna()
dbl_traite=dbl.loc[dbl.tmja==dbl.groupby('id_comptag').tmja.transform(max)].drop_duplicates().copy()
d87.df_attr_insert=pd.concat([dbl_traite,ss_dbl], axis=0, sort=False)
d87.df_attr_insert.pc_pl.fillna(-99, inplace=True)
annee='2018'
d87.df_attr_insert['dep']='87'
d87.df_attr_insert['reseau']='RD'
d87.df_attr_insert['gestionnai']='CD87'
d87.df_attr_insert['concession']='N'
d87.df_attr_insert['obs']=d87.df_attr_insert.apply(lambda x : f"""nouveau_point,{x['date_debut'].strftime("%d/%m/%Y")}-{x['date_fin'].strftime("%d/%m/%Y")}""" if not (pd.isnull(x['date_debut']) and  pd.isnull(x['date_fin'])) else None,axis=1)
d87.df_attr_insert.rename(columns={'absc' : 'abs', 'tmja':'tmja_'+annee,'pc_pl':'pc_pl_'+annee,'obs':'obs_'+annee},inplace=True)
d87.df_attr_insert.drop(['date_debut','date_fin','route'],axis=1,inplace=True)

#mettre à jour la geom
d87.maj_geom('gti_otv_pg11', 'comptage', 'na_2010_2018_p', dep='87')

In [944]:
Niort=MHCorbin(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\data\MHCorbin\rue de souche du 31 01 au 07 02 2020.mdb')

In [945]:
dfHoraireBddNiort=Niort.formaterDonneesHoraires(Niort.formaterDonneesIndiv(Niort.dfAgreg2Sens))

PasAssezMesureError: le fichier comporte moins de 7 jours complets de mesures. Nb jours complets : 6 

In [13]:
#récupérer un fichier depuis le mdb et en extraire les donnees horaires
Anglet=MHCorbin(r'C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\Anglet_ville\historique\2019\COMPTAGES ANGLET\Comptages 2019\donnees brutes\1.Av .des Dauphins.mdb')
dfHoraireBddAnglet=Anglet.formaterDonneesHoraires(Anglet.formaterDonneesIndiv(Anglet.dfAgreg2Sens))